In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/public_timeseries_testing_util.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras

In [3]:
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true != 0)|(y_pred != 0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)

In [4]:
s_clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')
s_clinical[['updrs_4']]=s_clinical[['updrs_4']].fillna(0)
s_clinical=s_clinical.dropna().reset_index(drop=True)
s_clinical

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,75_36,75,36,1.0,8.0,38.0,0.0,On
1,337_0,337,0,5.0,7.0,6.0,0.0,On
2,337_36,337,36,8.0,7.0,8.0,0.0,On
3,527_36,527,36,2.0,18.0,22.0,0.0,On
4,557_36,557,36,6.0,9.0,17.0,0.0,On
...,...,...,...,...,...,...,...,...
1115,65290_6,65290,6,1.0,12.0,22.0,0.0,On
1116,65290_12,65290,12,4.0,14.0,13.0,0.0,On
1117,65290_24,65290,24,2.0,13.0,14.0,2.0,On
1118,65290_30,65290,30,4.0,16.0,13.0,0.0,On


In [5]:
proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
print('Proteins shape:',proteins.shape)
proteins.head()

Proteins shape: (232741, 5)


,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [6]:
peptides = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv')
print('Peptides shape:', peptides.shape)
peptides.head()

Peptides shape: (981834, 6)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [7]:
clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
print('Clinical shape:', clinical.shape)
clinical.head()

Clinical shape: (2615, 8)


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [8]:
# Merge the proteins data and peptides data on the common columns.
merged_proteins_peptides = pd.merge(proteins, peptides, on = ['visit_id', 'visit_month', 'patient_id', 'UniProt'])

# Merge the merged protein-peptides data with the clinical data on the common columns.
merged = pd.merge(merged_proteins_peptides, clinical, on = ['visit_id', 'visit_month', 'patient_id'])

# Show the merged data.
merged

,visit_id,visit_month,patient_id,UniProt,NPX,Peptide,PeptideAbundance,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,0,55,O00391,11254.3,NEQEQPLGQWHLS,11254.30,10.0,6.0,15.0,NaN,NaN
1,55_0,0,55,O00533,732430.0,GNPEPTFSWTK,102060.00,10.0,6.0,15.0,NaN,NaN
2,55_0,0,55,O00533,732430.0,IEIPSSVQQVPTIIK,174185.00,10.0,6.0,15.0,NaN,NaN
3,55_0,0,55,O00533,732430.0,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.90,10.0,6.0,15.0,NaN,NaN
4,55_0,0,55,O00533,732430.0,SMEQNGPGLEYR,30838.70,10.0,6.0,15.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
941739,58648_108,108,58648,Q9UHG2,369437.0,ILAGSADSEGVAAPR,202820.00,6.0,0.0,0.0,NaN,NaN
941740,58648_108,108,58648,Q9UKV8,105830.0,SGNIPAGTTVDTK,105830.00,6.0,0.0,0.0,NaN,NaN
941741,58648_108,108,58648,Q9Y646,21257.6,LALLVDTVGPR,21257.60,6.0,0.0,0.0,NaN,NaN
941742,58648_108,108,58648,Q9Y6R7,17953.1,AGC(UniMod_4)VAESTAVC(UniMod_4)R,5127.26,6.0,0.0,0.0,NaN,NaN


In [9]:
# Pivot the data.
pivoted = merged.pivot(index = 'visit_id', columns = ['Peptide'], values = 'PeptideAbundance')

# See the pivoted data.
pivoted

Peptide,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,AATVGSLAGQPLQER,AAVYHHFISDGVR,ADDKETC(UniMod_4)FAEEGK,ADDKETC(UniMod_4)FAEEGKK,ADDLGKGGNEESTKTGNAGSR,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,6580710.0,31204.4,7735070.0,NaN,NaN,NaN,46620.3,236144.0,NaN,NaN,...,202274.0,NaN,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,NaN,7207.30
10053_12,6333510.0,52277.6,5394390.0,NaN,NaN,NaN,57554.5,108298.0,45885.4,NaN,...,201009.0,NaN,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
10053_18,7129640.0,61522.0,7011920.0,35984.7,17188.00,19787.3,36029.4,708729.0,5067790.0,30838.2,...,220728.0,NaN,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
10138_12,7404780.0,46107.2,10610900.0,NaN,20910.20,66662.3,55253.9,79575.5,6201210.0,26720.0,...,188362.0,9433.71,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
10138_24,13788300.0,56910.3,6906160.0,13785.5,11004.20,63672.7,36819.8,34160.9,2117430.0,15645.2,...,206187.0,6365.15,3521800.0,69984.6,496737.0,80919.3,111799.0,NaN,56977.6,4903.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,6312970.0,44462.7,12455000.0,11051.3,1163.18,43279.8,67743.5,325328.0,4666550.0,11038.5,...,289888.0,8615.27,8770410.0,33599.1,926094.0,118897.0,133682.0,571879.0,80268.3,54889.70
942_12,11289900.0,46111.7,11297300.0,NaN,13894.10,53755.0,40289.3,565112.0,NaN,26495.8,...,173259.0,4767.63,374307.0,35767.3,250397.0,65966.9,77976.8,486239.0,45032.7,NaN
942_24,10161900.0,32145.0,12388000.0,25869.2,17341.80,48625.5,45223.9,84448.0,4684800.0,23150.2,...,185428.0,5554.53,NaN,64049.8,479473.0,68505.7,74483.1,561398.0,52916.4,21847.60


In [10]:

merged2 = merged.groupby(by=['visit_id','UniProt']).mean().reset_index()
merged2

,visit_id,UniProt,visit_month,patient_id,NPX,PeptideAbundance,updrs_1,updrs_2,updrs_3,updrs_4
0,10053_0,O00391,0.0,10053.0,9104.27,9104.270000,3.0,0.0,13.0,0.0
1,10053_0,O00533,0.0,10053.0,402321.00,57474.485714,3.0,0.0,13.0,0.0
2,10053_0,O14773,0.0,10053.0,7150.57,7150.570000,3.0,0.0,13.0,0.0
3,10053_0,O14791,0.0,10053.0,2497.84,2497.840000,3.0,0.0,13.0,0.0
4,10053_0,O15240,0.0,10053.0,83002.90,41501.450000,3.0,0.0,13.0,0.0
...,...,...,...,...,...,...,...,...,...,...
223263,942_6,Q9UHG2,6.0,942.0,306621.00,76655.122500,8.0,2.0,21.0,NaN
223264,942_6,Q9UKV8,6.0,942.0,82335.50,82335.500000,8.0,2.0,21.0,NaN
223265,942_6,Q9UNU6,6.0,942.0,24018.70,24018.700000,8.0,2.0,21.0,NaN
223266,942_6,Q9Y646,6.0,942.0,18939.50,18939.500000,8.0,2.0,21.0,NaN


In [11]:
# Pivot the data.
pivoted2 = merged2.pivot(index = 'visit_id', columns = ['UniProt'], values = 'NPX').reset_index()

# See the pivoted data.
pivoted2

UniProt,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
0,10053_0,9104.27,402321.0,NaN,NaN,7150.57,2497.84,83002.9,15113.6,167327.0,...,NaN,9469.45,94237.6,NaN,23016.0,177983.0,65900.0,15382.0,NaN,19017.40
1,10053_12,10464.20,435586.0,NaN,NaN,NaN,NaN,197117.0,15099.1,164268.0,...,NaN,14408.40,NaN,NaN,28537.0,171733.0,65668.1,NaN,9295.65,25697.80
2,10053_18,13235.70,507386.0,7126.96,24525.7,NaN,2372.71,126506.0,16289.6,168107.0,...,317477.0,38667.20,111107.0,NaN,37932.6,245188.0,59986.1,10813.3,NaN,29102.70
3,10138_12,12600.20,494581.0,9165.06,27193.5,22506.10,6015.90,156313.0,54546.4,204013.0,...,557904.0,44556.90,155619.0,14647.90,36927.7,229232.0,106564.0,26077.7,21441.80,7642.42
4,10138_24,12003.20,522138.0,4498.51,17189.8,29112.40,2665.15,151169.0,52338.1,240892.0,...,NaN,47836.70,177619.0,17061.10,25510.4,176722.0,59471.4,12639.2,15091.40,6168.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,8699_24,9983.00,400290.0,24240.10,NaN,16943.50,6303.17,77493.6,46435.3,254247.0,...,NaN,25690.60,NaN,6859.82,19106.7,121161.0,113872.0,14413.9,28225.50,8062.07
1064,942_12,6757.32,360858.0,18367.60,14760.7,18603.40,1722.77,86847.4,37741.3,212132.0,...,45742.3,33518.60,94049.7,13415.70,21324.7,234094.0,82410.4,19183.7,17804.10,12277.00
1065,942_24,NaN,352722.0,22834.90,23393.1,16693.50,1487.91,114772.0,36095.7,185836.0,...,180475.0,29770.60,95949.9,11344.40,23637.6,256654.0,76931.9,19168.2,19215.90,14625.60
1066,942_48,11627.80,251820.0,22046.50,26360.5,22440.20,2117.43,82241.9,30146.6,167633.0,...,197987.0,29283.80,121696.0,19169.80,16724.9,232301.0,96905.9,21120.9,14089.80,16418.50


In [12]:
# Add visit_month, the 4 scores, and medication status.
df = pd.merge(clinical, pivoted, on = 'visit_id', how = 'right').set_index('visit_id')
df = pd.merge(df, pivoted2, on = 'visit_id', how = 'outer').set_index('visit_id')
df

,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,10053,0,3.0,0.0,13.0,0.0,NaN,6580710.0,31204.4,7735070.0,...,NaN,9469.45,94237.6,NaN,23016.0,177983.0,65900.0,15382.0,NaN,19017.40
10053_12,10053,12,4.0,2.0,8.0,0.0,NaN,6333510.0,52277.6,5394390.0,...,NaN,14408.40,NaN,NaN,28537.0,171733.0,65668.1,NaN,9295.65,25697.80
10053_18,10053,18,2.0,2.0,0.0,0.0,NaN,7129640.0,61522.0,7011920.0,...,317477.0,38667.20,111107.0,NaN,37932.6,245188.0,59986.1,10813.3,NaN,29102.70
10138_12,10138,12,3.0,6.0,31.0,0.0,On,7404780.0,46107.2,10610900.0,...,557904.0,44556.90,155619.0,14647.90,36927.7,229232.0,106564.0,26077.7,21441.80,7642.42
10138_24,10138,24,4.0,7.0,19.0,10.0,On,13788300.0,56910.3,6906160.0,...,NaN,47836.70,177619.0,17061.10,25510.4,176722.0,59471.4,12639.2,15091.40,6168.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,8699,24,11.0,10.0,13.0,2.0,On,6312970.0,44462.7,12455000.0,...,NaN,25690.60,NaN,6859.82,19106.7,121161.0,113872.0,14413.9,28225.50,8062.07
942_12,942,12,5.0,2.0,25.0,0.0,NaN,11289900.0,46111.7,11297300.0,...,45742.3,33518.60,94049.7,13415.70,21324.7,234094.0,82410.4,19183.7,17804.10,12277.00
942_24,942,24,2.0,3.0,23.0,NaN,NaN,10161900.0,32145.0,12388000.0,...,180475.0,29770.60,95949.9,11344.40,23637.6,256654.0,76931.9,19168.2,19215.90,14625.60


In [13]:
# Insert the visit_month column to the desired position.
df.insert(6, 'visit_month', df.pop('visit_month'))
df

,patient_id,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,visit_month,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,10053,3.0,0.0,13.0,0.0,NaN,0,6580710.0,31204.4,7735070.0,...,NaN,9469.45,94237.6,NaN,23016.0,177983.0,65900.0,15382.0,NaN,19017.40
10053_12,10053,4.0,2.0,8.0,0.0,NaN,12,6333510.0,52277.6,5394390.0,...,NaN,14408.40,NaN,NaN,28537.0,171733.0,65668.1,NaN,9295.65,25697.80
10053_18,10053,2.0,2.0,0.0,0.0,NaN,18,7129640.0,61522.0,7011920.0,...,317477.0,38667.20,111107.0,NaN,37932.6,245188.0,59986.1,10813.3,NaN,29102.70
10138_12,10138,3.0,6.0,31.0,0.0,On,12,7404780.0,46107.2,10610900.0,...,557904.0,44556.90,155619.0,14647.90,36927.7,229232.0,106564.0,26077.7,21441.80,7642.42
10138_24,10138,4.0,7.0,19.0,10.0,On,24,13788300.0,56910.3,6906160.0,...,NaN,47836.70,177619.0,17061.10,25510.4,176722.0,59471.4,12639.2,15091.40,6168.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,8699,11.0,10.0,13.0,2.0,On,24,6312970.0,44462.7,12455000.0,...,NaN,25690.60,NaN,6859.82,19106.7,121161.0,113872.0,14413.9,28225.50,8062.07
942_12,942,5.0,2.0,25.0,0.0,NaN,12,11289900.0,46111.7,11297300.0,...,45742.3,33518.60,94049.7,13415.70,21324.7,234094.0,82410.4,19183.7,17804.10,12277.00
942_24,942,2.0,3.0,23.0,NaN,NaN,24,10161900.0,32145.0,12388000.0,...,180475.0,29770.60,95949.9,11344.40,23637.6,256654.0,76931.9,19168.2,19215.90,14625.60


In [14]:
df = df.drop('patient_id', axis = 1)
df

,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,visit_month,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,3.0,0.0,13.0,0.0,NaN,0,6580710.0,31204.4,7735070.0,NaN,...,NaN,9469.45,94237.6,NaN,23016.0,177983.0,65900.0,15382.0,NaN,19017.40
10053_12,4.0,2.0,8.0,0.0,NaN,12,6333510.0,52277.6,5394390.0,NaN,...,NaN,14408.40,NaN,NaN,28537.0,171733.0,65668.1,NaN,9295.65,25697.80
10053_18,2.0,2.0,0.0,0.0,NaN,18,7129640.0,61522.0,7011920.0,35984.7,...,317477.0,38667.20,111107.0,NaN,37932.6,245188.0,59986.1,10813.3,NaN,29102.70
10138_12,3.0,6.0,31.0,0.0,On,12,7404780.0,46107.2,10610900.0,NaN,...,557904.0,44556.90,155619.0,14647.90,36927.7,229232.0,106564.0,26077.7,21441.80,7642.42
10138_24,4.0,7.0,19.0,10.0,On,24,13788300.0,56910.3,6906160.0,13785.5,...,NaN,47836.70,177619.0,17061.10,25510.4,176722.0,59471.4,12639.2,15091.40,6168.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,11.0,10.0,13.0,2.0,On,24,6312970.0,44462.7,12455000.0,11051.3,...,NaN,25690.60,NaN,6859.82,19106.7,121161.0,113872.0,14413.9,28225.50,8062.07
942_12,5.0,2.0,25.0,0.0,NaN,12,11289900.0,46111.7,11297300.0,NaN,...,45742.3,33518.60,94049.7,13415.70,21324.7,234094.0,82410.4,19183.7,17804.10,12277.00
942_24,2.0,3.0,23.0,NaN,NaN,24,10161900.0,32145.0,12388000.0,25869.2,...,180475.0,29770.60,95949.9,11344.40,23637.6,256654.0,76931.9,19168.2,19215.90,14625.60


In [15]:
# Replace NaN with 0 in the Peptides columns.
# df.loc[:, 'AADDTWEPFASGK':] = df.loc[:, 'AADDTWEPFASGK':].fillna(0)
# df

In [16]:
# Drop upd23b_clinical_state_on_medication column from the previous merged train dataset df.
df = df.drop('upd23b_clinical_state_on_medication', axis = 1)
df

,updrs_1,updrs_2,updrs_3,updrs_4,visit_month,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,3.0,0.0,13.0,0.0,0,6580710.0,31204.4,7735070.0,NaN,NaN,...,NaN,9469.45,94237.6,NaN,23016.0,177983.0,65900.0,15382.0,NaN,19017.40
10053_12,4.0,2.0,8.0,0.0,12,6333510.0,52277.6,5394390.0,NaN,NaN,...,NaN,14408.40,NaN,NaN,28537.0,171733.0,65668.1,NaN,9295.65,25697.80
10053_18,2.0,2.0,0.0,0.0,18,7129640.0,61522.0,7011920.0,35984.7,17188.00,...,317477.0,38667.20,111107.0,NaN,37932.6,245188.0,59986.1,10813.3,NaN,29102.70
10138_12,3.0,6.0,31.0,0.0,12,7404780.0,46107.2,10610900.0,NaN,20910.20,...,557904.0,44556.90,155619.0,14647.90,36927.7,229232.0,106564.0,26077.7,21441.80,7642.42
10138_24,4.0,7.0,19.0,10.0,24,13788300.0,56910.3,6906160.0,13785.5,11004.20,...,NaN,47836.70,177619.0,17061.10,25510.4,176722.0,59471.4,12639.2,15091.40,6168.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,11.0,10.0,13.0,2.0,24,6312970.0,44462.7,12455000.0,11051.3,1163.18,...,NaN,25690.60,NaN,6859.82,19106.7,121161.0,113872.0,14413.9,28225.50,8062.07
942_12,5.0,2.0,25.0,0.0,12,11289900.0,46111.7,11297300.0,NaN,13894.10,...,45742.3,33518.60,94049.7,13415.70,21324.7,234094.0,82410.4,19183.7,17804.10,12277.00
942_24,2.0,3.0,23.0,NaN,24,10161900.0,32145.0,12388000.0,25869.2,17341.80,...,180475.0,29770.60,95949.9,11344.40,23637.6,256654.0,76931.9,19168.2,19215.90,14625.60


In [17]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

In [18]:
peptides_list = ['QGVNDNEEGFFSAR','AGLAASLAGPHSIVGR','MELERPGGNEITR','KTSLEDFYLDEER','RYIETDPANRDR','EWVAIESDSVQPVPR','LQDLYSIVR','AIQLTYNPDESSKPNMIDAATLK','GAYPLSIEPIGVR']
parkinson_proteins2 = (['O00533'
,'O14498'
,'O15240',
'O15394',
'O43505',
'O60888',
'P00738',
'P01034',
'P01042',
'P01717',
'P02452',
'P02649',
'P02751',
'P02753',
'P02787',
'P04075',
'P04156',
'P04180',
'P04216',
'P05060',
'P05067','visit_id'])

In [19]:
real_cols = [col for col in pivoted2.columns if col not in parkinson_proteins2]

In [20]:
parkinson_proteins = ['O14773', 'P02647','P07333','P00450','P10451','P01033','P01008','P01024','Q92876']

In [21]:
df[parkinson_proteins].isnull().sum().sort_values(ascending=False)

P07333    184
O14773     64
P01033     16
Q92876      9
P10451      2
P00450      1
P01008      1
P02647      0
P01024      0
dtype: int64

In [22]:
df3 = df[['updrs_1','updrs_2','updrs_3','updrs_4','visit_month']+parkinson_proteins+peptides_list]

In [23]:
df4 = df3.drop(columns=['updrs_3','updrs_4']).fillna(method='bfill')

In [24]:
df4=df4.fillna(method='ffill')

In [25]:
df4.isnull().sum()

updrs_1                    0
updrs_2                    0
visit_month                0
O14773                     0
P02647                     0
P07333                     0
P00450                     0
P10451                     0
P01033                     0
P01008                     0
P01024                     0
Q92876                     0
QGVNDNEEGFFSAR             0
AGLAASLAGPHSIVGR           0
MELERPGGNEITR              0
KTSLEDFYLDEER              0
RYIETDPANRDR               0
EWVAIESDSVQPVPR            0
LQDLYSIVR                  0
AIQLTYNPDESSKPNMIDAATLK    0
GAYPLSIEPIGVR              0
dtype: int64

In [26]:
df4[['updrs_3','updrs_4']]=df3[['updrs_3','updrs_4']]
df4

,updrs_1,updrs_2,visit_month,O14773,P02647,P07333,P00450,P10451,P01033,P01008,...,AGLAASLAGPHSIVGR,MELERPGGNEITR,KTSLEDFYLDEER,RYIETDPANRDR,EWVAIESDSVQPVPR,LQDLYSIVR,AIQLTYNPDESSKPNMIDAATLK,GAYPLSIEPIGVR,updrs_3,updrs_4
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,3.0,0.0,0,7150.57,4032650.0,23030.6,774736.0,193303.0,79917.7,1849090.0,...,101257.0,109732.0,152498.0,19098.8,56216.9,9830.31,50049.9,188379.0,13.0,0.0
10053_12,4.0,2.0,12,22506.10,5111760.0,23030.6,1025080.0,208776.0,78499.7,1769370.0,...,132973.0,160483.0,117543.0,19098.8,47869.2,9830.31,51032.8,387892.0,8.0,0.0
10053_18,2.0,2.0,18,22506.10,5713180.0,23030.6,1104540.0,429418.0,87682.5,2821870.0,...,164045.0,143219.0,185164.0,19098.8,65454.0,24608.50,60411.0,399025.0,0.0,0.0
10138_12,3.0,6.0,12,22506.10,22825800.0,18930.3,1116740.0,357973.0,86710.8,3265820.0,...,119883.0,308855.0,197340.0,12206.2,59907.3,21425.60,71845.3,371030.0,31.0,0.0
10138_24,4.0,7.0,24,29112.40,11136600.0,22585.5,1318710.0,302432.0,131876.0,2085790.0,...,134179.0,129009.0,262287.0,12206.2,59706.9,21425.60,82343.8,448570.0,19.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,11.0,10.0,24,16943.50,24721500.0,55226.3,1222820.0,242755.0,67859.8,3007210.0,...,95773.8,325202.0,90343.3,11499.3,62929.3,13324.40,108381.0,344304.0,13.0,2.0
942_12,5.0,2.0,12,18603.40,13923400.0,17834.3,868038.0,193181.0,24685.9,2268890.0,...,77477.3,97814.2,186355.0,17255.4,58234.4,13324.40,33180.7,241980.0,25.0,0.0
942_24,2.0,3.0,24,16693.50,15753400.0,19963.5,1124980.0,177773.0,85652.9,2498100.0,...,77477.3,118337.0,148678.0,17255.4,42143.8,11941.40,47029.1,344475.0,23.0,NaN


In [27]:
df4[['updrs_4']]=df4[['updrs_4']].fillna(0)

In [28]:
df4=df4.dropna()

In [29]:
df4

,updrs_1,updrs_2,visit_month,O14773,P02647,P07333,P00450,P10451,P01033,P01008,...,AGLAASLAGPHSIVGR,MELERPGGNEITR,KTSLEDFYLDEER,RYIETDPANRDR,EWVAIESDSVQPVPR,LQDLYSIVR,AIQLTYNPDESSKPNMIDAATLK,GAYPLSIEPIGVR,updrs_3,updrs_4
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,3.0,0.0,0,7150.57,4032650.0,23030.6,774736.0,193303.0,79917.7,1849090.0,...,101257.0,109732.0,152498.0,19098.8,56216.9,9830.31,50049.9,188379.0,13.0,0.0
10053_12,4.0,2.0,12,22506.10,5111760.0,23030.6,1025080.0,208776.0,78499.7,1769370.0,...,132973.0,160483.0,117543.0,19098.8,47869.2,9830.31,51032.8,387892.0,8.0,0.0
10053_18,2.0,2.0,18,22506.10,5713180.0,23030.6,1104540.0,429418.0,87682.5,2821870.0,...,164045.0,143219.0,185164.0,19098.8,65454.0,24608.50,60411.0,399025.0,0.0,0.0
10138_12,3.0,6.0,12,22506.10,22825800.0,18930.3,1116740.0,357973.0,86710.8,3265820.0,...,119883.0,308855.0,197340.0,12206.2,59907.3,21425.60,71845.3,371030.0,31.0,0.0
10138_24,4.0,7.0,24,29112.40,11136600.0,22585.5,1318710.0,302432.0,131876.0,2085790.0,...,134179.0,129009.0,262287.0,12206.2,59706.9,21425.60,82343.8,448570.0,19.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,11.0,10.0,24,16943.50,24721500.0,55226.3,1222820.0,242755.0,67859.8,3007210.0,...,95773.8,325202.0,90343.3,11499.3,62929.3,13324.40,108381.0,344304.0,13.0,2.0
942_12,5.0,2.0,12,18603.40,13923400.0,17834.3,868038.0,193181.0,24685.9,2268890.0,...,77477.3,97814.2,186355.0,17255.4,58234.4,13324.40,33180.7,241980.0,25.0,0.0
942_24,2.0,3.0,24,16693.50,15753400.0,19963.5,1124980.0,177773.0,85652.9,2498100.0,...,77477.3,118337.0,148678.0,17255.4,42143.8,11941.40,47029.1,344475.0,23.0,0.0


In [30]:
scaler=RobustScaler()
df5 = df4.drop(columns=['updrs_1','updrs_2','updrs_3','updrs_4'])
df5mean=df5.mean()
df5 = pd.DataFrame(scaler.fit_transform(df5),columns=df5.columns).set_index(df4.index)
df5

,visit_month,O14773,P02647,P07333,P00450,P10451,P01033,P01008,P01024,Q92876,QGVNDNEEGFFSAR,AGLAASLAGPHSIVGR,MELERPGGNEITR,KTSLEDFYLDEER,RYIETDPANRDR,EWVAIESDSVQPVPR,LQDLYSIVR,AIQLTYNPDESSKPNMIDAATLK,GAYPLSIEPIGVR
visit_id,,,,,,,,,,,,,,,,,,,
10053_0,-0.571429,-1.073314,-1.248081,-0.464741,-0.797496,-0.689007,0.006883,-1.135623,-0.895669,-0.389747,0.180677,-0.504401,-0.558325,-0.142207,0.324520,-0.163399,-1.099216,-0.087643,-1.127835
10053_12,-0.285714,0.589746,-1.055486,-0.464741,0.168980,-0.602641,-0.042523,-1.273565,-0.160894,-0.468043,1.470260,-0.001870,0.192047,-0.580900,0.324520,-0.403566,-1.099216,-0.051440,0.672645
10053_18,-0.142857,0.589746,-0.948147,-0.464741,0.475742,0.628919,0.277426,0.547606,0.230464,0.168895,1.470260,0.490458,-0.063207,0.267759,0.324520,0.102357,0.315503,0.293992,0.773114
10138_12,-0.285714,0.589746,2.106041,-0.667808,0.522842,0.230134,0.243569,1.315785,0.812829,0.019690,1.470260,-0.209277,2.385779,0.420571,-0.727541,-0.057224,0.010803,0.715157,0.520476
10138_24,0.000000,1.305232,0.019802,-0.486784,1.302565,-0.079880,1.817223,-0.726054,1.125155,0.843386,1.087510,0.017239,-0.273307,1.235672,-0.727541,-0.062990,0.010803,1.101854,1.220226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,0.000000,-0.012704,2.444378,1.129753,0.932373,-0.412980,-0.413240,0.868305,2.407334,-0.743990,1.979493,-0.591281,2.627476,-0.922263,-0.835439,0.029720,-0.764726,2.060895,0.279291
942_12,-0.285714,0.167069,0.517178,-0.722088,-0.437295,-0.689688,-1.917513,-0.409231,-0.545714,-0.716613,-0.815565,-0.881184,-0.734533,0.282707,0.043150,-0.105354,-0.764726,-0.708995,-0.644119
942_24,0.000000,-0.039780,0.843789,-0.616639,0.554653,-0.775691,0.206710,-0.012623,0.419344,-0.746518,-0.422266,-0.881184,-0.431097,-0.190149,0.043150,-0.568288,-0.897121,-0.198910,0.280834


In [31]:
ran=100


In [32]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer
import numpy as np
import warnings
warnings.filterwarnings('ignore')
#x=final_df.drop(columns=['visit_id','updrs_1','updrs_2','updrs_3','updrs_4'])
x=df5
y=df4.updrs_3
X_train,x_val,y_train,y_val = train_test_split(x,y, test_size=0.2,shuffle=True,random_state=100)
# 모델 리스트와 그리드 서치에 사용할 하이퍼파라미터 리스트
models = [LGBMRegressor(random_state=ran), RandomForestRegressor(random_state=ran),XGBRegressor(random_state=100),Lasso(random_state=100)]#Lasso()


params = [{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [2, 4, 8,10], 'min_samples_leaf': list(range(1,11,2))},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          #{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]}]
          {'alpha':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]}]
# SMAPE를 Scikit-learn에서 사용 가능한 형태로 변환하는 함수
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
def smape_plus_1(y_true, y_pred):
    y_true_plus_1 = y_true + 1
    y_pred_plus_1 = y_pred + 1
    metric = np.zeros(len(y_true_plus_1))
    
    numerator = np.abs(y_true_plus_1 - y_pred_plus_1)
    denominator = ((np.abs(y_true_plus_1) + np.abs(y_pred_plus_1)) / 2)
    
    mask_not_zeros = (y_true_plus_1 != 0) | (y_pred_plus_1 != 0)
    metric[mask_not_zeros] = numerator[mask_not_zeros] / denominator[mask_not_zeros]
    
    return 100 * np.nanmean(metric)    
    return 100 * np.mean(smap)
scorer = make_scorer(smape_plus_1, greater_is_better=False)
# 각 모델에 대한 그리드 서치와 OOF 예측 수행
models_3 = {}
oof_preds3 = {}
cv = KFold(n_splits=5, shuffle=True, random_state=123)
for i, model in enumerate(models):
    grid_search = GridSearchCV(model, params[i], scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
    grid_search.fit(X_train, y_train)
    best_model3 = grid_search.best_estimator_
    models_3[i] = best_model3
    oof_pred = np.zeros(len(y_train))
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        trn_X, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
        val_X, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
        best_model3.fit(trn_X, trn_y)
        oof_pred[val_idx] = best_model3.predict(val_X)
        smape_score = smape_plus_1(val_y, oof_pred[val_idx])
        print(f"Model {i+1}, Fold {fold_idx+1}, SMAPE: {smape_score:.2f}")
    oof_preds3[i] = oof_pred
    print("Best Parameters:", grid_search.best_params_)
    print("Best SMAPE:", -grid_search.best_score_)

Model 1, Fold 1, SMAPE: 76.78
Model 1, Fold 2, SMAPE: 82.04
Model 1, Fold 3, SMAPE: 80.65
Model 1, Fold 4, SMAPE: 78.12
Model 1, Fold 5, SMAPE: 76.76
Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100}
Best SMAPE: 78.86834415398582
Model 2, Fold 1, SMAPE: 76.47
Model 2, Fold 2, SMAPE: 82.83
Model 2, Fold 3, SMAPE: 81.45
Model 2, Fold 4, SMAPE: 77.38
Model 2, Fold 5, SMAPE: 77.40
Best Parameters: {'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 500}
Best SMAPE: 79.10389408836382
Model 3, Fold 1, SMAPE: 74.48
Model 3, Fold 2, SMAPE: 81.50
Model 3, Fold 3, SMAPE: 77.70
Model 3, Fold 4, SMAPE: 78.99
Model 3, Fold 5, SMAPE: 76.21
Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 500}
Best SMAPE: 77.77508933103199
Model 4, Fold 1, SMAPE: 78.16
Model 4, Fold 2, SMAPE: 83.75
Model 4, Fold 3, SMAPE: 82.32
Model 4, Fold 4, SMAPE: 77.26
Model 4, Fold 5, SMAPE: 80.30
Best Parameters: {'alpha': 0.1}
Best SMAPE: 80.359628

In [33]:
x3=x
y3=y

In [34]:
combinations=[[0.1, 0.3, 0.3, 0.3], [0.1, 0.2, 0.3, 0.4], [0.1, 0.1, 0.5, 0.3], [0.2, 0.2, 0.2, 0.4], [0.1, 0.1, 0.2, 0.6], [0.2, 0.3, 0.1, 0.4], [0.1, 0.2, 0.1, 0.6], [0.3, 0.2, 0.3, 0.2], [0.1, 0.4, 0.2, 0.3], [0.2, 0.2, 0.4, 0.2], [0.1, 0.1, 0.3, 0.5], [0.2, 0.4, 0.1, 0.3], [0.2, 0.1, 0.3, 0.4], [0.3, 0.3, 0.2, 0.2], [0.1, 0.1, 0.1, 0.7], [0.1, 0.5, 0.2, 0.2], [0.3, 0.1, 0.1, 0.5], [0.3, 0.4, 0.2, 0.1], [0.4, 0.1, 0.1, 0.4], [0.4, 0.2, 0.1, 0.3], [0.2, 0.3, 0.4, 0.1], [0.1, 0.1, 0.4, 0.4], [0.1, 0.2, 0.5, 0.2], [0.2, 0.2, 0.1, 0.5], [0.3, 0.3, 0.3, 0.1], [0.1, 0.1, 0.6, 0.2], [0.2, 0.1, 0.2, 0.5], [0.3, 0.1, 0.3,0.3]]

In [35]:
class EnsembleModel(BaseEstimator, RegressorMixin, TransformerMixin):
    
    def __init__(self, models, weights):
        self.models = models
        self.weights = weights
    
    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)
    
    def predict(self, X):
        pred = np.zeros(len(X))
        for i, model in enumerate(self.models):
            pred += self.weights[i] * model.predict(X)
        return pred


# 앙상블 모델에 대한 그리드 서치를 수행하며 가중치 튜닝 수행
models_ensemble = [models_3[0], models_3[1], models_3[2], models_3[3]]
params_ensemble = {'weights': combinations}
ensemble_model = EnsembleModel(models_ensemble, weights=[0.25, 0.25, 0.25, 0.25])
grid_search_ensemble = GridSearchCV(ensemble_model, params_ensemble, scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
grid_search_ensemble.fit(X_train, y_train)

best_weights = grid_search_ensemble.best_params_['weights']
best_ensemble_model3 = EnsembleModel(models_ensemble, weights=best_weights)
best_ensemble_model3.fit(X_train, y_train)
y_pred_ensemble = best_ensemble_model3.predict(x_val)
smape_ensemble = smape_plus_1(y_val, y_pred_ensemble)
print("Best Weights:", best_weights)
print("Best Ensemble SMAPE:",smape_ensemble)

Best Weights: [0.1, 0.1, 0.6, 0.2]
Best Ensemble SMAPE: 77.28064783508299


In [36]:
scaler1=RobustScaler()
df6 = df4.drop(columns=['updrs_1','updrs_2','updrs_4'])
df6 = pd.DataFrame(scaler1.fit_transform(df6),columns=df6.columns).set_index(df4.index)
df6

,visit_month,O14773,P02647,P07333,P00450,P10451,P01033,P01008,P01024,Q92876,QGVNDNEEGFFSAR,AGLAASLAGPHSIVGR,MELERPGGNEITR,KTSLEDFYLDEER,RYIETDPANRDR,EWVAIESDSVQPVPR,LQDLYSIVR,AIQLTYNPDESSKPNMIDAATLK,GAYPLSIEPIGVR,updrs_3
visit_id,,,,,,,,,,,,,,,,,,,,
10053_0,-0.571429,-1.073314,-1.248081,-0.464741,-0.797496,-0.689007,0.006883,-1.135623,-0.895669,-0.389747,0.180677,-0.504401,-0.558325,-0.142207,0.324520,-0.163399,-1.099216,-0.087643,-1.127835,-0.115385
10053_12,-0.285714,0.589746,-1.055486,-0.464741,0.168980,-0.602641,-0.042523,-1.273565,-0.160894,-0.468043,1.470260,-0.001870,0.192047,-0.580900,0.324520,-0.403566,-1.099216,-0.051440,0.672645,-0.307692
10053_18,-0.142857,0.589746,-0.948147,-0.464741,0.475742,0.628919,0.277426,0.547606,0.230464,0.168895,1.470260,0.490458,-0.063207,0.267759,0.324520,0.102357,0.315503,0.293992,0.773114,-0.615385
10138_12,-0.285714,0.589746,2.106041,-0.667808,0.522842,0.230134,0.243569,1.315785,0.812829,0.019690,1.470260,-0.209277,2.385779,0.420571,-0.727541,-0.057224,0.010803,0.715157,0.520476,0.576923
10138_24,0.000000,1.305232,0.019802,-0.486784,1.302565,-0.079880,1.817223,-0.726054,1.125155,0.843386,1.087510,0.017239,-0.273307,1.235672,-0.727541,-0.062990,0.010803,1.101854,1.220226,0.115385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,0.000000,-0.012704,2.444378,1.129753,0.932373,-0.412980,-0.413240,0.868305,2.407334,-0.743990,1.979493,-0.591281,2.627476,-0.922263,-0.835439,0.029720,-0.764726,2.060895,0.279291,-0.115385
942_12,-0.285714,0.167069,0.517178,-0.722088,-0.437295,-0.689688,-1.917513,-0.409231,-0.545714,-0.716613,-0.815565,-0.881184,-0.734533,0.282707,0.043150,-0.105354,-0.764726,-0.708995,-0.644119,0.346154
942_24,0.000000,-0.039780,0.843789,-0.616639,0.554653,-0.775691,0.206710,-0.012623,0.419344,-0.746518,-0.422266,-0.881184,-0.431097,-0.190149,0.043150,-0.568288,-0.897121,-0.198910,0.280834,0.269231


In [37]:

x=df6
y=df4.updrs_1
X_train,x_val,y_train,y_val = train_test_split(x,y, test_size=0.2,shuffle=True,random_state=100)
# 모델 리스트와 그리드 서치에 사용할 하이퍼파라미터 리스트
models = [LGBMRegressor(random_state=ran), RandomForestRegressor(random_state=ran),XGBRegressor(random_state=100),Lasso(random_state=100)]#Lasso()


params = [{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [2, 4, 8,10], 'min_samples_leaf': list(range(1,11,2))},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          #{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]}]
          {'alpha':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]}]
# SMAPE를 Scikit-learn에서 사용 가능한 형태로 변환하는 함수
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)
scorer = make_scorer(smape_plus_1, greater_is_better=False)
# 각 모델에 대한 그리드 서치와 OOF 예측 수행
models_1 = {}
oof_preds1 = {}
cv = KFold(n_splits=5, shuffle=True, random_state=100)
for i, model in enumerate(models):
    grid_search = GridSearchCV(model, params[i], scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
    grid_search.fit(X_train, y_train)
    best_model1 = grid_search.best_estimator_
    models_1[i] = best_model1
    oof_pred = np.zeros(len(y_train))
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        trn_X, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
        val_X, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
        best_model1.fit(trn_X, trn_y)
        oof_pred[val_idx] = best_model1.predict(val_X)
        smape_score = smape_plus_1(val_y, oof_pred[val_idx])
        print(f"Model {i+1}, Fold {fold_idx+1}, SMAPE: {smape_score:.2f}")
    oof_preds1[i] = oof_pred
    print("Best Parameters:", grid_search.best_params_)
    print("Best SMAPE:", -grid_search.best_score_)

Model 1, Fold 1, SMAPE: 50.99
Model 1, Fold 2, SMAPE: 49.99
Model 1, Fold 3, SMAPE: 50.81
Model 1, Fold 4, SMAPE: 53.06
Model 1, Fold 5, SMAPE: 55.01
Best Parameters: {'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 100}
Best SMAPE: 51.97394145080075
Model 2, Fold 1, SMAPE: 51.24
Model 2, Fold 2, SMAPE: 50.69
Model 2, Fold 3, SMAPE: 48.45
Model 2, Fold 4, SMAPE: 54.06
Model 2, Fold 5, SMAPE: 54.87
Best Parameters: {'max_depth': 7, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 300}
Best SMAPE: 51.86110363150059
Model 3, Fold 1, SMAPE: 49.95
Model 3, Fold 2, SMAPE: 50.25
Model 3, Fold 3, SMAPE: 51.71
Model 3, Fold 4, SMAPE: 53.32
Model 3, Fold 5, SMAPE: 53.91
Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 300}
Best SMAPE: 51.8284232546374
Model 4, Fold 1, SMAPE: 52.72
Model 4, Fold 2, SMAPE: 53.11
Model 4, Fold 3, SMAPE: 50.05
Model 4, Fold 4, SMAPE: 54.94
Model 4, Fold 5, SMAPE: 54.94
Best Parameters: {'alpha': 0.07}
Best SMAPE: 53.15062

In [38]:
models_ensemble = [models_1[0], models_1[1], models_1[2], models_1[3]]
params_ensemble = {'weights': combinations}
ensemble_model = EnsembleModel(models_ensemble, weights=[0.25, 0.25, 0.25, 0.25])
grid_search_ensemble = GridSearchCV(ensemble_model, params_ensemble, scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
grid_search_ensemble.fit(X_train, y_train)

best_weights = grid_search_ensemble.best_params_['weights']
best_ensemble_model1 = EnsembleModel(models_ensemble, weights=best_weights)
best_ensemble_model1.fit(X_train, y_train)
y_pred_ensemble = best_ensemble_model1.predict(x_val)
smape_ensemble = smape_plus_1(y_val, y_pred_ensemble)
print("Best Weights:", best_weights)
print("Best Ensemble SMAPE:",smape_ensemble)

Best Weights: [0.2, 0.3, 0.4, 0.1]
Best Ensemble SMAPE: 48.03255676840623


In [39]:
scaler2=RobustScaler()
df7 = df4.drop(columns=['updrs_2','updrs_4'])
df7 = pd.DataFrame(scaler2.fit_transform(df7),columns=df7.columns).set_index(df4.index)
df7

,updrs_1,visit_month,O14773,P02647,P07333,P00450,P10451,P01033,P01008,P01024,...,QGVNDNEEGFFSAR,AGLAASLAGPHSIVGR,MELERPGGNEITR,KTSLEDFYLDEER,RYIETDPANRDR,EWVAIESDSVQPVPR,LQDLYSIVR,AIQLTYNPDESSKPNMIDAATLK,GAYPLSIEPIGVR,updrs_3
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,-0.428571,-0.571429,-1.073314,-1.248081,-0.464741,-0.797496,-0.689007,0.006883,-1.135623,-0.895669,...,0.180677,-0.504401,-0.558325,-0.142207,0.324520,-0.163399,-1.099216,-0.087643,-1.127835,-0.115385
10053_12,-0.285714,-0.285714,0.589746,-1.055486,-0.464741,0.168980,-0.602641,-0.042523,-1.273565,-0.160894,...,1.470260,-0.001870,0.192047,-0.580900,0.324520,-0.403566,-1.099216,-0.051440,0.672645,-0.307692
10053_18,-0.571429,-0.142857,0.589746,-0.948147,-0.464741,0.475742,0.628919,0.277426,0.547606,0.230464,...,1.470260,0.490458,-0.063207,0.267759,0.324520,0.102357,0.315503,0.293992,0.773114,-0.615385
10138_12,-0.428571,-0.285714,0.589746,2.106041,-0.667808,0.522842,0.230134,0.243569,1.315785,0.812829,...,1.470260,-0.209277,2.385779,0.420571,-0.727541,-0.057224,0.010803,0.715157,0.520476,0.576923
10138_24,-0.285714,0.000000,1.305232,0.019802,-0.486784,1.302565,-0.079880,1.817223,-0.726054,1.125155,...,1.087510,0.017239,-0.273307,1.235672,-0.727541,-0.062990,0.010803,1.101854,1.220226,0.115385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,0.714286,0.000000,-0.012704,2.444378,1.129753,0.932373,-0.412980,-0.413240,0.868305,2.407334,...,1.979493,-0.591281,2.627476,-0.922263,-0.835439,0.029720,-0.764726,2.060895,0.279291,-0.115385
942_12,-0.142857,-0.285714,0.167069,0.517178,-0.722088,-0.437295,-0.689688,-1.917513,-0.409231,-0.545714,...,-0.815565,-0.881184,-0.734533,0.282707,0.043150,-0.105354,-0.764726,-0.708995,-0.644119,0.346154
942_24,-0.571429,0.000000,-0.039780,0.843789,-0.616639,0.554653,-0.775691,0.206710,-0.012623,0.419344,...,-0.422266,-0.881184,-0.431097,-0.190149,0.043150,-0.568288,-0.897121,-0.198910,0.280834,0.269231


In [40]:

x=df7
y=df4.updrs_2
X_train,x_val,y_train,y_val = train_test_split(x,y, test_size=0.2,shuffle=True,random_state=100)
# 모델 리스트와 그리드 서치에 사용할 하이퍼파라미터 리스트
models = [LGBMRegressor(random_state=ran), RandomForestRegressor(random_state=ran),XGBRegressor(random_state=100),Lasso(random_state=100)]#Lasso()


params = [{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [2, 4, 8,10], 'min_samples_leaf': list(range(1,11,2))},
          {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]},
          #{'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.05, 0.1], 'max_depth': [3, 5, 7,9,11]}]
          {'alpha':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]}]
# SMAPE를 Scikit-learn에서 사용 가능한 형태로 변환하는 함수
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)
scorer = make_scorer(smape_plus_1, greater_is_better=False)
# 각 모델에 대한 그리드 서치와 OOF 예측 수행
models_2 = {}
oof_preds2 = {}
cv = KFold(n_splits=5, shuffle=True, random_state=100)
for i, model in enumerate(models):
    grid_search = GridSearchCV(model, params[i], scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
    grid_search.fit(X_train, y_train)
    best_model2 = grid_search.best_estimator_
    models_2[i] = best_model2
    oof_pred = np.zeros(len(y_train))
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        trn_X, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
        val_X, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
        best_model2.fit(trn_X, trn_y)
        oof_pred[val_idx] = best_model2.predict(val_X)
        smape_score = smape_plus_1(val_y, oof_pred[val_idx])
        print(f"Model {i+1}, Fold {fold_idx+1}, SMAPE: {smape_score:.2f}")
    oof_preds2[i] = oof_pred
    print("Best Parameters:", grid_search.best_params_)
    print("Best SMAPE:", -grid_search.best_score_)

Model 1, Fold 1, SMAPE: 40.60
Model 1, Fold 2, SMAPE: 36.65
Model 1, Fold 3, SMAPE: 35.09
Model 1, Fold 4, SMAPE: 44.30
Model 1, Fold 5, SMAPE: 35.96
Best Parameters: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 500}
Best SMAPE: 38.519886061910725
Model 2, Fold 1, SMAPE: 38.39
Model 2, Fold 2, SMAPE: 35.13
Model 2, Fold 3, SMAPE: 37.12
Model 2, Fold 4, SMAPE: 43.87
Model 2, Fold 5, SMAPE: 36.79
Best Parameters: {'max_depth': 7, 'min_samples_leaf': 7, 'min_samples_split': 2, 'n_estimators': 300}
Best SMAPE: 38.25886848076841
Model 3, Fold 1, SMAPE: 38.65
Model 3, Fold 2, SMAPE: 37.64
Model 3, Fold 3, SMAPE: 39.00
Model 3, Fold 4, SMAPE: 42.41
Model 3, Fold 5, SMAPE: 38.00
Best Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 300}
Best SMAPE: 39.13866374269233
Model 4, Fold 1, SMAPE: 48.47
Model 4, Fold 2, SMAPE: 40.21
Model 4, Fold 3, SMAPE: 42.65
Model 4, Fold 4, SMAPE: 48.33
Model 4, Fold 5, SMAPE: 46.37
Best Parameters: {'alpha': 0.1}
Best SMAPE: 45.2054

In [41]:

models_ensemble = [models_2[0], models_2[1], models_2[2], models_2[3]]
params_ensemble = {'weights': combinations}
ensemble_model = EnsembleModel(models_ensemble, weights=[0.25, 0.25, 0.25, 0.25])
grid_search_ensemble = GridSearchCV(ensemble_model, params_ensemble, scoring=scorer, cv=cv, n_jobs=-1,verbose=0)
grid_search_ensemble.fit(X_train, y_train)

best_weights = grid_search_ensemble.best_params_['weights']
best_ensemble_model2 = EnsembleModel(models_ensemble, weights=best_weights)
best_ensemble_model2.fit(X_train, y_train)
y_pred_ensemble = best_ensemble_model2.predict(x_val)
smape_ensemble = smape_plus_1(y_val, y_pred_ensemble)
print("Best Weights:", best_weights)
print("Best Ensemble SMAPE:",smape_ensemble)

Best Weights: [0.3, 0.3, 0.2, 0.2]
Best Ensemble SMAPE: 36.07741298094232


In [42]:
merged

,visit_id,visit_month,patient_id,UniProt,NPX,Peptide,PeptideAbundance,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,0,55,O00391,11254.3,NEQEQPLGQWHLS,11254.30,10.0,6.0,15.0,NaN,NaN
1,55_0,0,55,O00533,732430.0,GNPEPTFSWTK,102060.00,10.0,6.0,15.0,NaN,NaN
2,55_0,0,55,O00533,732430.0,IEIPSSVQQVPTIIK,174185.00,10.0,6.0,15.0,NaN,NaN
3,55_0,0,55,O00533,732430.0,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.90,10.0,6.0,15.0,NaN,NaN
4,55_0,0,55,O00533,732430.0,SMEQNGPGLEYR,30838.70,10.0,6.0,15.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
941739,58648_108,108,58648,Q9UHG2,369437.0,ILAGSADSEGVAAPR,202820.00,6.0,0.0,0.0,NaN,NaN
941740,58648_108,108,58648,Q9UKV8,105830.0,SGNIPAGTTVDTK,105830.00,6.0,0.0,0.0,NaN,NaN
941741,58648_108,108,58648,Q9Y646,21257.6,LALLVDTVGPR,21257.60,6.0,0.0,0.0,NaN,NaN
941742,58648_108,108,58648,Q9Y6R7,17953.1,AGC(UniMod_4)VAESTAVC(UniMod_4)R,5127.26,6.0,0.0,0.0,NaN,NaN


In [43]:
# Merge the merged protein-peptides data with the clinical data on the common columns.
merged4 = pd.merge(s_clinical, clinical, on = ['visit_id', 'visit_month', 'patient_id','updrs_1','updrs_2','updrs_3','updrs_4'],how='outer')
merged4.drop(columns=['upd23b_clinical_state_on_medication_x','upd23b_clinical_state_on_medication_y'],inplace=True)
merged4

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4
0,75_36,75,36,1.0,8.0,38.0,0.0
1,337_0,337,0,5.0,7.0,6.0,0.0
2,337_36,337,36,8.0,7.0,8.0,0.0
3,527_36,527,36,2.0,18.0,22.0,0.0
4,557_36,557,36,6.0,9.0,17.0,0.0
...,...,...,...,...,...,...,...
3730,65043_48,65043,48,7.0,6.0,13.0,0.0
3731,65043_54,65043,54,4.0,8.0,11.0,1.0
3732,65043_60,65043,60,6.0,6.0,16.0,1.0
3733,65043_72,65043,72,3.0,9.0,14.0,1.0


In [44]:
merged4[['updrs_4']]=merged4[['updrs_4']].fillna(0)
merged4=merged4.dropna()
merged4

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4
0,75_36,75,36,1.0,8.0,38.0,0.0
1,337_0,337,0,5.0,7.0,6.0,0.0
2,337_36,337,36,8.0,7.0,8.0,0.0
3,527_36,527,36,2.0,18.0,22.0,0.0
4,557_36,557,36,6.0,9.0,17.0,0.0
...,...,...,...,...,...,...,...
3730,65043_48,65043,48,7.0,6.0,13.0,0.0
3731,65043_54,65043,54,4.0,8.0,11.0,1.0
3732,65043_60,65043,60,6.0,6.0,16.0,1.0
3733,65043_72,65043,72,3.0,9.0,14.0,1.0


In [45]:
sss= set(sorted(merged4['visit_month'].values))

In [46]:
sss

{0, 3, 6, 9, 12, 18, 24, 30, 36, 42, 48, 54, 60, 72, 84, 96, 108}

In [47]:
dfs = {}
for j in sss:
    dfs['df{}'.format(j)] = merged4[merged4['visit_month']==j]

            

In [48]:
# patient_id 기준으로 묶어서 각 visit_month에 대한 updrs 값들을 열로 갖는 데이터프레임 생성
df_grouped1 = merged4.groupby('patient_id').apply(lambda x: x.set_index('visit_month')['updrs_1'].rename_axis(None)).unstack().reset_index()
df_grouped2 = merged4.groupby('patient_id').apply(lambda x: x.set_index('visit_month')['updrs_2'].rename_axis(None)).unstack().reset_index()
df_grouped3 = merged4.groupby('patient_id').apply(lambda x: x.set_index('visit_month')['updrs_3'].rename_axis(None)).unstack().reset_index()
df_grouped4 = merged4.groupby('patient_id').apply(lambda x: x.set_index('visit_month')['updrs_4'].rename_axis(None)).unstack().reset_index()

In [49]:
df_grouped3

,patient_id,0,3,6,9,12,18,24,30,36,42,48,54,60,72,84,96,108
0,55,15.0,25.0,34.0,30.0,41.0,38.0,49.0,49.0,51.0,41.0,52.0,51.0,56.0,NaN,NaN,NaN,NaN
1,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,337,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,64938,16.0,NaN,20.0,NaN,21.0,NaN,21.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
714,65006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
715,65043,16.0,19.0,19.0,11.0,14.0,13.0,NaN,4.0,5.0,15.0,13.0,11.0,16.0,14.0,20.0,NaN,NaN
716,65290,18.0,NaN,22.0,NaN,13.0,NaN,14.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold

dataset_directory = Path('/kaggle/input/amp-parkinsons-disease-progression-prediction')
sys.path.append(dataset_directory)
import amp_pd_peptide

In [51]:
df_train_clinical_data = pd.read_csv(dataset_directory / 'train_clinical_data.csv')
print(f'Train Clinical Data Shape: {df_train_clinical_data.shape} - Memory Usage: {df_train_clinical_data.memory_usage().sum() / 1024 ** 2:.2f} MB')

patient_count = df_train_clinical_data['patient_id'].nunique()
visit_count_mean = df_train_clinical_data.groupby('patient_id')['visit_month'].count().mean()
print(f'Patient Count: {patient_count} - Mean Visit Count: {visit_count_mean:.2f}')

Train Clinical Data Shape: (2615, 8) - Memory Usage: 0.16 MB
Patient Count: 248 - Mean Visit Count: 10.54


In [52]:
df_supplemental_clinical_data = pd.read_csv(dataset_directory / 'supplemental_clinical_data.csv')
print(f'Supplemental Clinical Data Shape: {df_supplemental_clinical_data.shape} - Memory Usage: {df_supplemental_clinical_data.memory_usage().sum() / 1024 ** 2:.2f} MB')

patient_count = df_supplemental_clinical_data['patient_id'].nunique()
visit_count_mean = df_supplemental_clinical_data.groupby('patient_id')['visit_month'].count().mean()
print(f'Patient Count: {patient_count} - Mean Visit Count: {visit_count_mean:.2f}')

Supplemental Clinical Data Shape: (2223, 8) - Memory Usage: 0.14 MB
Patient Count: 771 - Mean Visit Count: 2.88


In [53]:
target_columns_clinical_data = ['updrs_1']
target_columns_clinical_and_supplemental_data = ['updrs_2', 'updrs_3', 'updrs_4']

target_visit_month_medians_clinical_data = df_train_clinical_data.groupby('visit_month')[target_columns_clinical_data].median()
target_visit_month_medians_clinical_and_supplemental_data = pd.concat((
    df_train_clinical_data,
    df_supplemental_clinical_data
), axis=0).groupby('visit_month')[target_columns_clinical_and_supplemental_data].median()

# Drop 5th month visit that is coming from the supplemental clinical data
target_visit_month_medians_clinical_and_supplemental_data = target_visit_month_medians_clinical_and_supplemental_data.drop(5)

# Concatenate visit_month medians of targets
target_visit_month_medians = pd.concat((
    target_visit_month_medians_clinical_data,
    target_visit_month_medians_clinical_and_supplemental_data
), axis=1, ignore_index=False)

# Replace expanding window max of updrs values with current updrs values
target_visit_month_medians = target_visit_month_medians.expanding(min_periods=1).max()
target_visit_month_medians

,updrs_1,updrs_2,updrs_3,updrs_4
visit_month,,,,
0,4.5,4.0,18.0,0.0
3,4.5,5.0,19.0,0.0
6,6.0,6.0,21.0,0.0
9,6.0,6.0,21.0,0.0
12,6.0,6.0,21.0,0.0
18,6.0,6.0,21.0,0.0
24,6.0,6.0,21.0,0.0
30,7.0,6.0,22.0,0.0
36,7.0,6.0,22.0,0.0


In [54]:
tt1=target_visit_month_medians.reset_index()
smm=pd.DataFrame({'visit_month':[66,78,90],'updrs_1':[7,7,7],'updrs_2':[9,9,9],'updrs_3':[26,29,29],'updrs_4':[0,2,3]})
target_visit_month_medians2=pd.concat([tt1,smm])
target_visit_month_medians2=target_visit_month_medians2.set_index('visit_month')
#target_visit_month_medians2.drop(columns=['visit_month'],inplace=True)
target_visit_month_medians2

,updrs_1,updrs_2,updrs_3,updrs_4
visit_month,,,,
0,4.5,4.0,18.0,0.0
3,4.5,5.0,19.0,0.0
6,6.0,6.0,21.0,0.0
9,6.0,6.0,21.0,0.0
12,6.0,6.0,21.0,0.0
18,6.0,6.0,21.0,0.0
24,6.0,6.0,21.0,0.0
30,7.0,6.0,22.0,0.0
36,7.0,6.0,22.0,0.0


In [55]:
for i in range(len(df_grouped1)):
    for j in df_grouped1.columns[1:]:
        if pd.isna(df_grouped1.loc[i,j]):
            df_grouped1.loc[i,j]=target_visit_month_medians.loc[j,'updrs_1']


In [56]:
df_grouped1[[15,21,27,33,66,78,90,102,114]]=np.nan
df_sorted1 = df_grouped1.drop(columns=['patient_id']).reindex(columns=sorted(df_grouped1.drop(columns=['patient_id']).columns, key=int))
df_sorted1.loc[:,15]=df_sorted1.loc[:,12]
df_sorted1.loc[:,21]=df_sorted1.loc[:,18]
df_sorted1.loc[:,27]=df_sorted1.loc[:,24]
df_sorted1.loc[:,33]=df_sorted1.loc[:,30]
df_sorted1.loc[:,102]=df_sorted1.loc[:,96]
df_sorted1.loc[:,114]=df_sorted1.loc[:,108]
df_sorted1=df_sorted1.fillna(7)
df_sorted1['patient_id']=df_grouped1['patient_id']
df_sorted1

,0,3,6,9,12,15,18,21,24,27,...,66,72,78,84,90,96,102,108,114,patient_id
0,10.0,10.0,8.0,8.0,10.0,10.0,7.0,7.0,16.0,16.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,55
1,4.5,4.5,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,75
2,5.0,4.5,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,337
3,4.5,4.5,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,527
4,4.5,4.5,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,4.0,4.5,1.0,6.0,2.0,2.0,6.0,6.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,64938
714,4.5,4.5,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,65006
715,2.0,2.0,3.0,2.0,4.0,4.0,6.0,6.0,6.0,6.0,...,7.0,3.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,65043
716,4.0,4.5,1.0,6.0,4.0,4.0,6.0,6.0,2.0,2.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,65290


In [57]:
for i in range(len(df_grouped2)):
    for j in df_grouped2.columns[1:]:
        if pd.isna(df_grouped2.loc[i,j]):
            df_grouped2.loc[i,j]=target_visit_month_medians.loc[j,'updrs_2']
df_grouped2

,patient_id,0,3,6,9,12,18,24,30,36,42,48,54,60,72,84,96,108
0,55,6.0,7.0,10.0,9.0,10.0,13.0,9.0,13.0,18.0,20.0,16.0,18.0,21.0,9.0,9.0,9.0,9.0
1,75,4.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,8.0,7.0,7.0,9.0,9.0,9.0,9.0,9.0,9.0
2,337,7.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,7.0,7.0,7.0,9.0,9.0,9.0,9.0,9.0,9.0
3,527,4.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,18.0,7.0,7.0,9.0,9.0,9.0,9.0,9.0,9.0
4,557,4.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,9.0,7.0,7.0,9.0,9.0,9.0,9.0,9.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,64938,5.0,5.0,5.0,6.0,7.0,6.0,7.0,11.0,6.0,7.0,7.0,9.0,9.0,9.0,9.0,9.0,9.0
714,65006,4.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,20.0,7.0,7.0,9.0,9.0,9.0,9.0,9.0,9.0
715,65043,6.0,11.0,6.0,7.0,7.0,7.0,6.0,4.0,7.0,10.0,6.0,8.0,6.0,9.0,9.0,9.0,9.0
716,65290,14.0,5.0,12.0,6.0,14.0,6.0,13.0,16.0,6.0,7.0,7.0,9.0,9.0,9.0,9.0,9.0,9.0


In [58]:
df_grouped2[[15,21,27,33,66,78,90,102,114]]=np.nan
df_sorted2 = df_grouped2.drop(columns=['patient_id']).reindex(columns=sorted(df_grouped2.drop(columns=['patient_id']).columns, key=int))
df_sorted2.loc[:,15]=df_sorted2.loc[:,12]
df_sorted2.loc[:,21]=df_sorted2.loc[:,18]
df_sorted2.loc[:,27]=df_sorted2.loc[:,24]
df_sorted2.loc[:,33]=df_sorted2.loc[:,30]
df_sorted2.loc[:,102]=df_sorted2.loc[:,96]
df_sorted2.loc[:,114]=df_sorted2.loc[:,108]
df_sorted2=df_sorted2.fillna(9)
df_sorted2['patient_id']=df_grouped2['patient_id']
df_sorted2

,0,3,6,9,12,15,18,21,24,27,...,66,72,78,84,90,96,102,108,114,patient_id
0,6.0,7.0,10.0,9.0,10.0,10.0,13.0,13.0,9.0,9.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,55
1,4.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,75
2,7.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,337
3,4.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,527
4,4.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,5.0,5.0,5.0,6.0,7.0,7.0,6.0,6.0,7.0,7.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,64938
714,4.0,5.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,65006
715,6.0,11.0,6.0,7.0,7.0,7.0,7.0,7.0,6.0,6.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,65043
716,14.0,5.0,12.0,6.0,14.0,14.0,6.0,6.0,13.0,13.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,65290


In [59]:
for i in range(len(df_grouped3)):
    for j in df_grouped3.columns[1:]:
        if pd.isna(df_grouped3.loc[i,j]):
            df_grouped3.loc[i,j]=target_visit_month_medians.loc[j,'updrs_3']
df_grouped3

,patient_id,0,3,6,9,12,18,24,30,36,42,48,54,60,72,84,96,108
0,55,15.0,25.0,34.0,30.0,41.0,38.0,49.0,49.0,51.0,41.0,52.0,51.0,56.0,29.0,29.0,29.0,34.5
1,75,18.0,19.0,21.0,21.0,21.0,21.0,21.0,22.0,38.0,23.0,23.0,26.0,26.0,29.0,29.0,29.0,34.5
2,337,6.0,19.0,21.0,21.0,21.0,21.0,21.0,22.0,8.0,23.0,23.0,26.0,26.0,29.0,29.0,29.0,34.5
3,527,18.0,19.0,21.0,21.0,21.0,21.0,21.0,22.0,22.0,23.0,23.0,26.0,26.0,29.0,29.0,29.0,34.5
4,557,18.0,19.0,21.0,21.0,21.0,21.0,21.0,22.0,17.0,23.0,23.0,26.0,26.0,29.0,29.0,29.0,34.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,64938,16.0,19.0,20.0,21.0,21.0,21.0,21.0,22.0,22.0,23.0,23.0,26.0,26.0,29.0,29.0,29.0,34.5
714,65006,18.0,19.0,21.0,21.0,21.0,21.0,21.0,22.0,31.0,23.0,23.0,26.0,26.0,29.0,29.0,29.0,34.5
715,65043,16.0,19.0,19.0,11.0,14.0,13.0,21.0,4.0,5.0,15.0,13.0,11.0,16.0,14.0,20.0,29.0,34.5
716,65290,18.0,19.0,22.0,21.0,13.0,21.0,14.0,13.0,22.0,23.0,23.0,26.0,26.0,29.0,29.0,29.0,34.5


In [60]:
df_grouped3[[15,21,27,33,66,78,90,102,114]]=np.nan
df_sorted3 = df_grouped3.drop(columns=['patient_id']).reindex(columns=sorted(df_grouped3.drop(columns=['patient_id']).columns, key=int))
df_sorted3.loc[:,15]=df_sorted3.loc[:,12]
df_sorted3.loc[:,21]=df_sorted3.loc[:,18]
df_sorted3.loc[:,27]=df_sorted3.loc[:,24]
df_sorted3.loc[:,33]=df_sorted3.loc[:,30]
df_sorted3.loc[:,102]=df_sorted3.loc[:,96]
df_sorted3.loc[:,114]=df_sorted3.loc[:,108]
df_sorted3=df_sorted3.fillna(29)
df_sorted3['patient_id']=df_grouped3['patient_id']
df_sorted3

,0,3,6,9,12,15,18,21,24,27,...,66,72,78,84,90,96,102,108,114,patient_id
0,15.0,25.0,34.0,30.0,41.0,41.0,38.0,38.0,49.0,49.0,...,29.0,29.0,29.0,29.0,29.0,29.0,29.0,34.5,34.5,55
1,18.0,19.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,...,29.0,29.0,29.0,29.0,29.0,29.0,29.0,34.5,34.5,75
2,6.0,19.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,...,29.0,29.0,29.0,29.0,29.0,29.0,29.0,34.5,34.5,337
3,18.0,19.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,...,29.0,29.0,29.0,29.0,29.0,29.0,29.0,34.5,34.5,527
4,18.0,19.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,...,29.0,29.0,29.0,29.0,29.0,29.0,29.0,34.5,34.5,557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,16.0,19.0,20.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,...,29.0,29.0,29.0,29.0,29.0,29.0,29.0,34.5,34.5,64938
714,18.0,19.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,...,29.0,29.0,29.0,29.0,29.0,29.0,29.0,34.5,34.5,65006
715,16.0,19.0,19.0,11.0,14.0,14.0,13.0,13.0,21.0,21.0,...,29.0,14.0,29.0,20.0,29.0,29.0,29.0,34.5,34.5,65043
716,18.0,19.0,22.0,21.0,13.0,13.0,21.0,21.0,14.0,14.0,...,29.0,29.0,29.0,29.0,29.0,29.0,29.0,34.5,34.5,65290


In [61]:
for i in range(len(df_grouped4)):
    for j in df_grouped4.columns[1:]:
        if pd.isna(df_grouped4.loc[i,j]):
            df_grouped4.loc[i,j]=target_visit_month_medians.loc[j,'updrs_4']
df_grouped4

,patient_id,0,3,6,9,12,18,24,30,36,42,48,54,60,72,84,96,108
0,55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,4.0
1,75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,4.0
2,337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,4.0
3,527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,4.0
4,557,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,64938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,4.0
714,65006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,4.0
715,65043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,4.0,4.0
716,65290,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,4.0


In [62]:
df_grouped4[[15,21,27,33,66,78,90,102,114]]=np.nan
df_sorted4 = df_grouped4.drop(columns=['patient_id']).reindex(columns=sorted(df_grouped4.drop(columns=['patient_id']).columns, key=int))
df_sorted4.loc[:,15]=0
df_sorted4.loc[:,21]=0
df_sorted4.loc[:,27]=0
df_sorted4.loc[:,33]=0
df_sorted4[66]=0
df_sorted4[78]=2
df_sorted4[90]=3
df_sorted4[102]=4
df_sorted4[114]=4
df_sorted4['patient_id']=df_grouped4['patient_id']
df_sorted4

,0,3,6,9,12,15,18,21,24,27,...,66,72,78,84,90,96,102,108,114,patient_id
0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0,1.0,2,3.0,3,4.0,4,4.0,4,55
1,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0,1.0,2,3.0,3,4.0,4,4.0,4,75
2,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0,1.0,2,3.0,3,4.0,4,4.0,4,337
3,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0,1.0,2,3.0,3,4.0,4,4.0,4,527
4,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0,1.0,2,3.0,3,4.0,4,4.0,4,557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0,1.0,2,3.0,3,4.0,4,4.0,4,64938
714,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0,1.0,2,3.0,3,4.0,4,4.0,4,65006
715,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0,1.0,2,3.0,3,4.0,4,4.0,4,65043
716,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2.0,0,...,0,1.0,2,3.0,3,4.0,4,4.0,4,65290


In [63]:
index1 = [index for index in range(0,13,3)] +[index for index in range(18,85,6)]
index2 = [index for index in range(6,13,3)] +[index for index in range(18,97,6)]
index0 = [index for index in range(0,13,3)] +[index for index in range(18,73,6)]
index00= [index for index in range(84,97,6)]+[108]

In [64]:
def predictplus(aaaa,i,j,p):
    df100=pd.DataFrame()
    df100['p0'] = aaaa
    df100['p6'] = aaaa+i
    df100['p12'] = aaaa+i+j
    df100['p24'] = aaaa+i+j+p
    return df100

In [65]:
# df999=pd.DataFrame()
# visit=[]
# up2=[]
# up2_6=[]
# up2_12=[]
# up2_24=[]
# for i in range(len(df_sorted2)):
#     for j in df_sorted2.columns[:-1]:
#         if j%6==0 and j<=90:
#             visit.append(str(df_sorted2['patient_id'].loc[i])+'_'+str(j))
#             up2.append(df_sorted2.loc[i,j])
#             up2_6.append(df_sorted2.loc[i,j+6])
#             up2_12.append(df_sorted2.loc[i,j+12])
#             up2_24.append(df_sorted2.loc[i,j+24])
# df999['visit_id']=visit    
# df999['up2_0']=up2
# df999['up2_6']=up2_6
# df999['up2_12']=up2_12
# df999['up2_24']=up2_24

In [66]:
# df999[['patient_id', 'visit_month']] = df999['visit_id'].str.split('_', expand=True)
# df999

In [67]:
# # visit_id 열을 리스트로 추출
# visit_id_list = yyy['visit_id'].tolist()

# # df2에서 visit_id가 visit_id_list에 있는 행만 추출
# dfmonth2 = df999[df999['visit_id'].isin(visit_id_list)].reset_index(drop=True)
# dfmonth2


In [68]:
# visit_id_list2 = dfmonth2['visit_id'].tolist()

# # df2에서 visit_id가 visit_id_list에 있는 행만 추출
# x2 = x.reset_index()[x.reset_index()['visit_id'].isin(visit_id_list2)].reset_index(drop=True)

In [69]:
# x2 = x2.sort_values('visit_id')
# x2

In [70]:
# dfmonth2 = dfmonth2.sort_values('visit_id')
# dfmonth2

In [71]:

# aaa=[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
# sk=[]
# # Replace expanding window max of updrs values with current updrs values
# for i in aaa:
#     for j in aaa:
#         for k in aaa:
#             s=predictplus((best_ensemble_model2.predict(x2.drop(columns=['visit_id']))),i,j,k)
#             sk.append([(smape_plus_1(dfmonth2['up2_0'],s.iloc[:,0])+smape_plus_1(dfmonth2['up2_6'],s.iloc[:,1])+smape_plus_1(dfmonth2['up2_12'],s.iloc[:,2])+smape_plus_1(dfmonth2['up2_24'],s.iloc[:,3]))/4,(i,j,k)])


In [72]:
# sorted(sk)[0]

In [73]:
# df999=pd.DataFrame()
# visit=[]
# up1=[]
# up1_6=[]
# up1_12=[]
# up1_24=[]
# for i in range(len(df_sorted1)):
#     for j in df_sorted1.columns[:-1]:
#         if j%6==0 and j<=90:
#             visit.append(str(df_sorted1['patient_id'].loc[i])+'_'+str(j))
#             up1.append(df_sorted1.loc[i,j])
#             up1_6.append(df_sorted1.loc[i,j+6])
#             up1_12.append(df_sorted1.loc[i,j+12])
#             up1_24.append(df_sorted1.loc[i,j+24])
# df999['visit_id']=visit    
# df999['up1_0']=up2
# df999['up1_6']=up2_6
# df999['up1_12']=up2_12
# df999['up1_24']=up2_24
# df999[['patient_id', 'visit_month']] = df999['visit_id'].str.split('_', expand=True)
# df999
# visit_id 열을 리스트로 추출
# visit_id_list = yyy['visit_id'].tolist()

# # df2에서 visit_id가 visit_id_list에 있는 행만 추출
# dfmonth1 = df999[df999['visit_id'].isin(visit_id_list)].reset_index(drop=True)
# dfmonth1
# visit_id_list1 = dfmonth1['visit_id'].tolist()

# # df2에서 visit_id가 visit_id_list에 있는 행만 추출
# x1 = x.reset_index()[x.reset_index()['visit_id'].isin(visit_id_list1)].reset_index(drop=True)

In [74]:
# x1 = x1.sort_values('visit_id')
# dfmonth1 = dfmonth1.sort_values('visit_id')
# aaa=[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
# sk=[]
# # Replace expanding window max of updrs values with current updrs values
# for i in aaa:
#     for j in aaa:
#         for k in aaa:
#             s=predictplus((best_ensemble_model1.predict(x1.drop(columns=['visit_id']))),i,j,k)
#             sk.append([(smape_plus_1(dfmonth1['up1_0'],s.iloc[:,0])+smape_plus_1(dfmonth1['up1_6'],s.iloc[:,1])+smape_plus_1(dfmonth1['up1_12'],s.iloc[:,2])+smape_plus_1(dfmonth1['up1_24'],s.iloc[:,3]))/4,(i,j,k)])

# sorted(sk)[1]

In [75]:
cv1=[]
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(df_sorted2.iloc[:,0]), pd.DataFrame(df_sorted2.iloc[:,2]), test_size=0.2,random_state=i)

# LightGBM 모델을 초기화합니다.
    model021 = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20,random_state=1)

# 모델에 train 데이터를 학습시킵니다.
    model021.fit(X_train, y_train)

# 모델이 test 데이터를 예측한 결과를 출력합니다.
    y_pred = model021.predict(X_test)
    cv1.append([smape_plus_1(y_pred,y_test[6]),i])
sorted(cv1)[0]

[20.25767036040767, 52]

In [76]:
cv1=[]
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(df_sorted2.iloc[:,0]), pd.DataFrame(df_sorted2.iloc[:,4]), test_size=0.2,random_state=i)

# LightGBM 모델을 초기화합니다.
    model022 = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20,random_state=1)

# 모델에 train 데이터를 학습시킵니다.
    model022.fit(X_train, y_train)

# 모델이 test 데이터를 예측한 결과를 출력합니다.
    y_pred = model022.predict(X_test)
    cv1.append([smape_plus_1(y_pred,y_test[12]),i])
sorted(cv1)[0]

[27.680337364945935, 21]

### 

In [77]:
cv1=[]
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(df_sorted2.iloc[:,0]), pd.DataFrame(df_sorted2.iloc[:,8]), test_size=0.2,random_state=i)

# LightGBM 모델을 초기화합니다.
    model023 = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20,random_state=1)

# 모델에 train 데이터를 학습시킵니다.
    model023.fit(X_train, y_train)

# 모델이 test 데이터를 예측한 결과를 출력합니다.
    y_pred = model023.predict(X_test)
    cv1.append([smape_plus_1(y_pred,y_test[24]),i])
sorted(cv1)[0]

[30.59081212174063, 61]

In [78]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(df_sorted2.iloc[:,0]), pd.DataFrame(df_sorted2.iloc[:,2]), test_size=0.2,random_state=1)

# LightGBM 모델을 초기화합니다.
model021 = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20,random_state=52)

# 모델에 train 데이터를 학습시킵니다.
model021.fit(X_train, y_train)

# 모델이 test 데이터를 예측한 결과를 출력합니다.
y_pred = model021.predict(X_test)
print(smape_plus_1(y_pred,y_test[6]))
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(df_sorted2.iloc[:,0]), pd.DataFrame(df_sorted2.iloc[:,4]), test_size=0.2,random_state=1)

# LightGBM 모델을 초기화합니다.
model022 = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20,random_state=21)

# 모델에 train 데이터를 학습시킵니다.
model022.fit(X_train, y_train)

# 모델이 test 데이터를 예측한 결과를 출력합니다.
y_pred = model022.predict(X_test)
print(smape_plus_1(y_pred,y_test[12]))
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(df_sorted2.iloc[:,0]), pd.DataFrame(df_sorted2.iloc[:,8]), test_size=0.2,random_state=1)

# LightGBM 모델을 초기화합니다.
model023 = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20,random_state=61)

# 모델에 train 데이터를 학습시킵니다.
model023.fit(X_train, y_train)

# 모델이 test 데이터를 예측한 결과를 출력합니다.
y_pred = model023.predict(X_test)
print(smape_plus_1(y_pred,y_test[24]))

25.73336244103062
33.88514697308753
32.6790039486093


In [79]:
#X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(pd.concat([df_sorted1.iloc[:,0],df_sorted1.iloc[:,9]]), pd.DataFrame(df_sorted1.iloc[:,2]), test_size=0.2,random_state=14)

In [80]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(df_sorted1.iloc[:,0]), pd.DataFrame(df_sorted1.iloc[:,2]), test_size=0.2,random_state=14)

# LightGBM 모델을 초기화합니다.
model011 = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20,random_state=1)

# 모델에 train 데이터를 학습시킵니다.
model011.fit(X_train, y_train)

# 모델이 test 데이터를 예측한 결과를 출력합니다.
y_pred = model011.predict(X_test)
print(smape_plus_1(y_pred,y_test[6]))
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(df_sorted1.iloc[:,0]), pd.DataFrame(df_sorted1.iloc[:,4]), test_size=0.2,random_state=59)

# LightGBM 모델을 초기화합니다.
model012 = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20,random_state=1)

# 모델에 train 데이터를 학습시킵니다.
model012.fit(X_train, y_train)

# 모델이 test 데이터를 예측한 결과를 출력합니다.
y_pred = model012.predict(X_test)
print(smape_plus_1(y_pred,y_test[12]))
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(df_sorted1.iloc[:,0]), pd.DataFrame(df_sorted1.iloc[:,8]), test_size=0.2,random_state=77)

# LightGBM 모델을 초기화합니다.
model013 = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20,random_state=1)

# 모델에 train 데이터를 학습시킵니다.
model013.fit(X_train, y_train)

# 모델이 test 데이터를 예측한 결과를 출력합니다.
y_pred = model013.predict(X_test)
print(smape_plus_1(y_pred,y_test[24]))

20.409047602228053
23.37560792830624
26.557047629686604


In [81]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(df_sorted3.iloc[:,0]), pd.DataFrame(df_sorted3.iloc[:,2]), test_size=0.2,random_state=1)

# LightGBM 모델을 초기화합니다.
model031 = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20,random_state=52)

# 모델에 train 데이터를 학습시킵니다.
model031.fit(X_train, y_train)

# 모델이 test 데이터를 예측한 결과를 출력합니다.
y_pred = model031.predict(X_test)
print(smape_plus_1(y_pred,y_test[6]))
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(df_sorted3.iloc[:,0]), pd.DataFrame(df_sorted3.iloc[:,4]), test_size=0.2,random_state=1)

# LightGBM 모델을 초기화합니다.
model032 = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20,random_state=100)

# 모델에 train 데이터를 학습시킵니다.
model032.fit(X_train, y_train)

# 모델이 test 데이터를 예측한 결과를 출력합니다.
y_pred = model032.predict(X_test)
print(smape_plus_1(y_pred,y_test[12]))
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(df_sorted3.iloc[:,0]), pd.DataFrame(df_sorted3.iloc[:,8]), test_size=0.2,random_state=1)

# LightGBM 모델을 초기화합니다.
model033 = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20,random_state=61)

# 모델에 train 데이터를 학습시킵니다.
model033.fit(X_train, y_train)

# 모델이 test 데이터를 예측한 결과를 출력합니다.
y_pred = model033.predict(X_test)
print(smape_plus_1(y_pred,y_test[24]))

22.030679170208046
33.13271993547823
31.788116679358758


In [82]:
def mergeall(test, test_proteins, test_peptides):
    # Merge the proteins data and peptides data on the common columns.
    test_proteins.fillna(0)
    test_peptides.fillna(0)
    merged_proteins_peptides = pd.merge(test_proteins, test_peptides, on = ['visit_id', 'visit_month', 'patient_id', 'UniProt'])

# Merge the merged protein-peptides data with the clinical data on the common columns.
    merged =  merged_proteins_peptides
    dfsub = pd.DataFrame(columns=parkinson_proteins+peptides_list)
    pivoted = merged.pivot(index = 'visit_id', columns = ['Peptide'], values = 'PeptideAbundance')
    merged2 = merged.groupby(by=['visit_id','UniProt']).mean().reset_index()
    pivoted2 = merged2.pivot(index = 'visit_id', columns = ['UniProt'], values = 'NPX').reset_index()
# Add visit_month, the 4 scores, and medication status.
    df = pd.merge(pivoted, pivoted2, on = 'visit_id', how = 'outer').set_index('visit_id')
    for col in parkinson_proteins+peptides_list:
        if col in df.columns:
            dfsub[col] = df[col]
        else:
            dfsub[col] = np.nan
    dfsub=dfsub.fillna(df.mean())
    df4 = dfsub.fillna(method='ffill')
    df4=df4.fillna(method='bfill')
    df4=df4.fillna(df5mean)
    df4=df4.fillna(0)
    return df4
    

In [83]:
def get_predictions(my_train, test_proteins,test_peptides, model):
    # Forecast
    dfff=[]
    my_train = my_train.fillna(0)
    train_all303 = mergeall(my_train,test_proteins,test_peptides)
    ffinal = train_all303.reset_index()
    tempall=pd.DataFrame()
    tempall=my_train[['visit_id']]
    #ffinal = ffinal.rename(columns={'group_key_x': 'group_key'})
    #ffinal['visit_id'] = ffinal.index
    
   # dropcol=['visit_id','patient_id','UniProt','Peptide','row_id','updrs_test']
    dropcol=['visit_id']
   # df = ffinal[['visit_id']].drop_duplicates()
    df = ffinal[['visit_id']]
    df00 = my_train[['visit_id']]
    ffinal[['patient_id', 'visit_month']] = ffinal['visit_id'].str.split('_', expand=True)
    ffinal=ffinal.drop(columns=['visit_id','patient_id'])
    target=['updrs_3','updrs_1','updrs_2','updrs_4']
    for u in target:   
        # Here is where we will save the final results
        df['result_' + str(u)] = 0
        # Predict  
        if u == 'updrs_3':
            df5 = ffinal
            last_col = df5.iloc[:, -1]
            dfsi = df5.iloc[:, :-1]
            dfsi.insert(0, column='new_col_name', value=last_col)
            df5=dfsi
            df5 = pd.DataFrame(scaler.transform(dfsi),columns=dfsi.columns)
            df['result_' + str(u)] = best_ensemble_model3.predict(df5).tolist()
        if u == 'updrs_1':
            df55 = ffinal
            last_col = df55.iloc[:, -1]
            dfsib = df55.iloc[:, :-1]
            dfsib.insert(0, column='new_col_name', value=last_col)
            df55=dfsib
            df55['updrs_3']=df['result_updrs_3']
            df55 = pd.DataFrame(scaler1.transform(dfsib),columns=dfsib.columns)
            df['result_' + str(u)] = best_ensemble_model1.predict(df55).tolist()
        if u == 'updrs_2':
            df555 = ffinal
            last_col = df555.iloc[:, -1]
            dfsiba = df555.iloc[:, :-1]
            dfsiba.insert(0, column='new_col_name', value=last_col)
            dfsiba.rename(columns={'new_col_name': 'visit_month'}, inplace=True)
            df555=dfsiba
            df555['updrs_3']=df['result_updrs_3']
            df555['updrs_1']=df['result_updrs_1']
            last_col = df555.iloc[:, -1]
            dfsiba2 = df555.iloc[:, :-1]
            dfsiba2.insert(0, column='new_col_name', value=last_col)
            dfsiba2.rename(columns={'new_col_name': 'updrs_1'}, inplace=True)
            df555=dfsiba2
            df555 = pd.DataFrame(scaler2.transform(dfsiba2),columns=dfsiba2.columns)
            df['result_' + str(u)] = best_ensemble_model2.predict(df555).tolist()
        if u == 'updrs_4':
            df['result_' + str(u)] = 0
    
    a1=model031.predict(df[['result_updrs_3']])
    a2=model032.predict(df[['result_updrs_3']])
    a3=model033.predict(df[['result_updrs_3']])
    a4=model021.predict(df[['result_updrs_2']])
    a5=model022.predict(df[['result_updrs_2']])
    a6=model023.predict(df[['result_updrs_2']])
    a7=model011.predict(df[['result_updrs_1']])
    a8=model012.predict(df[['result_updrs_1']])
    a9=model013.predict(df[['result_updrs_1']])
    a=pd.DataFrame(columns=['dfupdrs3_6','updrs3_12','updrs3_24','updrs2_6','updrs2_12','updrs2_24','updrs1_6','updrs1_12','updrs1_24'])

    a['updrs1_0']=df[['result_updrs_1']]-1.5
    a['updrs1_6']=a7-1
    a['updrs1_12']=a8-1
    a['updrs1_24']=a9-1
    a['updrs2_0']=df[['result_updrs_2']]
    a['updrs2_6']=a4
    a['updrs2_12']=a5
    a['updrs2_24']=a6
    a['updrs3_0']=df[['result_updrs_3']]
    a['updrs3_6']=a1
    a['updrs3_12']=a2
    a['updrs3_24']=a3
            
 # Format for final submission
    result = pd.DataFrame()
    result2 = pd.DataFrame()
    for m in [0, 6, 12, 24]:
        for u in [1, 2, 3, 4]:
            temp = df[["visit_id", "result_updrs_" + str(u)]]
            temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
            if u!=4:
                temp["rating"] = a["updrs" + str(u)+"_"+str(m)]
            else:
                temp["rating"]=0
            temp = temp [['prediction_id', 'rating']]
            result = result.append(temp)   
            tempall["prediction_id"] = tempall["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
            result2=result2.append(tempall)
    result = result.drop_duplicates(subset=['prediction_id'])
    result=result.fillna(method='ffill')
    result=result.fillna(method='bfill')
    result=result.fillna(temp.mean())
    result=result.fillna(0)
    
    
    
    
    
    
    result2 = result2.drop_duplicates(subset=['prediction_id'])
    
    result2['rating']=np.nan
    visitlist=result[['prediction_id']].values.tolist()
    result2.loc[result2['prediction_id'].isin(result['prediction_id'].values.tolist()), 'rating'] = result['rating'].values
    result2[['patient_id', 'visit_month']] = result2['visit_id'].str.split('_', expand=True)
    for i in range(len(result2)):
        if pd.isna(result2[['rating']].iloc[i,:])[0] or int(result2[['rating']].iloc[i,:])<0 or int(result2[['rating']].iloc[i,:])>=50:
            if 'updrs_1' in result2['prediction_id'].iloc[i]:
                if result2[['prediction_id']].iloc[i][0][-8]==str(0) or result2[['prediction_id']].iloc[i][0][-8]==str(6):
                    result2['rating'].iloc[i]= target_visit_month_medians2.loc[int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_1']
                elif result2[['prediction_id']].iloc[i][0][-8]==str(2):
                    result2['rating'].iloc[i]= target_visit_month_medians2.loc[10+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_1']
                else:
                    result2['rating'].iloc[i]= target_visit_month_medians2.loc[20+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_1']
            if 'updrs_2' in result2['prediction_id'].iloc[i]:
                if result2[['prediction_id']].iloc[i][0][-8]==str(0) or result2[['prediction_id']].iloc[i][0][-8]==str(6):
                    result2['rating'].iloc[i]= target_visit_month_medians2.loc[int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_2']
                elif result2[['prediction_id']].iloc[i][0][-8]==str(2):
                    result2['rating'].iloc[i]= target_visit_month_medians2.loc[10+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_2']
                else:
                    result2['rating'].iloc[i]= target_visit_month_medians2.loc[20+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_2']
            if 'updrs_3' in result2['prediction_id'].iloc[i]:
                if result2[['prediction_id']].iloc[i][0][-8]==str(0) or result2[['prediction_id']].iloc[i][0][-8]==str(6):
                    result2['rating'].iloc[i]= target_visit_month_medians2.loc[int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_3']
                elif result2[['prediction_id']].iloc[i][0][-8]==str(2):
                    result2['rating'].iloc[i]= target_visit_month_medians2.loc[10+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_3']
                else:
                    result2['rating'].iloc[i]= target_visit_month_medians2.loc[20+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_3']
            if 'updrs_4' in result2['prediction_id'].iloc[i]:
                if result2[['prediction_id']].iloc[i][0][-8]==str(0) or result2[['prediction_id']].iloc[i][0][-8]==str(6):
                    result2['rating'].iloc[i]= target_visit_month_medians2.loc[int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_4']
                elif result2[['prediction_id']].iloc[i][0][-8]==str(2):
                    result2['rating'].iloc[i]= target_visit_month_medians2.loc[10+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_4']
                else:
                    result2['rating'].iloc[i]= target_visit_month_medians2.loc[20+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_4']
    result2=result2.fillna(method='ffill')
    result2=result2.fillna(method='bfill')
    result2=result2.fillna(temp.mean())
    result2=result2.fillna(0)
    result2 = result2[['prediction_id', 'rating']]
    visitlist22=result[['prediction_id']].values.tolist()
    result2 = result2.drop_duplicates(subset=['prediction_id'])
  #  for i in range(len(result2)):
    #    if (type(result2[['rating']].iloc[i,:][0]) == str:
     #       result2.loc[i,'rating']=0
    result2['rating'] = np.trunc(result2['rating'].astype(float))
    #import numpy as np
# div0, 음수, 문자 또는 100을 초과하는 값을 0으로 대체
    result2.loc[(result2['rating'] < 0) | (result2['rating'] > 60), 'rating'] = int(0)
    #result2['rating'] = np.where((result2['rating'] <= 0) | (result2['rating'] > 60) | (result2['rating'] == 'div0') | (result2['rating'].astype(str).str.isalpha()), int(0), result2['rating'])
    #result2.loc[result2['prediction_id'].isin(result['prediction_id'].values.tolist()), 'rating'] = result['rating'].values
   # for i in range(len(result2)):
        #if (type(result2[['rating']].iloc[i,:][0]) == str:
           # result2.loc[i,'rating']=0
    return result2

In [84]:
import pandas as pd
a=pd.DataFrame({'predict':['a_1','a_2','a_3'],'rating':['0',10,-10]})
a

,predict,rating
0,a_1,0
1,a_2,10
2,a_3,-10


In [85]:
for i in range(len(a)):
    if (type(a[['rating']].iloc[i,:][0]) == str):
        a.loc[i,'rating']=20
a

,predict,rating
0,a_1,20
1,a_2,10
2,a_3,-10


In [86]:
import amp_pd_peptide
env = amp_pd_peptide.make_env()
iter_test = env.iter_test()

In [87]:
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    display(test)
    
    
    display(mergeall(test,test_proteins,test_peptides))    
    result = get_predictions(test,test_proteins,test_peptides, model)

    #result=result.sort_values(by='prediction_id').reset_index(drop=True)
    env.predict(result)  

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


,visit_id,visit_month,patient_id,updrs_test,row_id
0,3342_0,0,3342,updrs_1,3342_0_updrs_1
1,3342_0,0,3342,updrs_2,3342_0_updrs_2
2,3342_0,0,3342,updrs_3,3342_0_updrs_3
3,3342_0,0,3342,updrs_4,3342_0_updrs_4
4,50423_0,0,50423,updrs_1,50423_0_updrs_1
5,50423_0,0,50423,updrs_2,50423_0_updrs_2
6,50423_0,0,50423,updrs_3,50423_0_updrs_3
7,50423_0,0,50423,updrs_4,50423_0_updrs_4


,O14773,P02647,P07333,P00450,P10451,P01033,P01008,P01024,Q92876,QGVNDNEEGFFSAR,AGLAASLAGPHSIVGR,MELERPGGNEITR,KTSLEDFYLDEER,RYIETDPANRDR,EWVAIESDSVQPVPR,LQDLYSIVR,AIQLTYNPDESSKPNMIDAATLK,GAYPLSIEPIGVR
visit_id,,,,,,,,,,,,,,,,,,
50423_0,16486.6,7490760.0,30788.5,964392.0,537918.0,105886.0,2688970.0,4395120.0,1457830.0,57492.1,221066.0,145250.0,192849.0,17305.925274,81796.4,19436.8,38107.0,268266.0


,visit_id,visit_month,patient_id,updrs_test,row_id
0,3342_6,6,3342,updrs_1,3342_6_updrs_1
1,3342_6,6,3342,updrs_2,3342_6_updrs_2
2,3342_6,6,3342,updrs_3,3342_6_updrs_3
3,3342_6,6,3342,updrs_4,3342_6_updrs_4
4,50423_6,6,50423,updrs_1,50423_6_updrs_1
5,50423_6,6,50423,updrs_2,50423_6_updrs_2
6,50423_6,6,50423,updrs_3,50423_6_updrs_3
7,50423_6,6,50423,updrs_4,50423_6_updrs_4


,O14773,P02647,P07333,P00450,P10451,P01033,P01008,P01024,Q92876,QGVNDNEEGFFSAR,AGLAASLAGPHSIVGR,MELERPGGNEITR,KTSLEDFYLDEER,RYIETDPANRDR,EWVAIESDSVQPVPR,LQDLYSIVR,AIQLTYNPDESSKPNMIDAATLK,GAYPLSIEPIGVR
visit_id,,,,,,,,,,,,,,,,,,
3342_6,14959.2,9133330.0,36292.179055,1159120.0,490139.0,81524.540548,2497440.0,3909780.0,1364170.0,73692.5,96942.1,168874.0,204589.0,17305.925274,101192.0,28540.8,71852.4,369823.0


In [88]:
get_predictions(test,test_proteins,test_peptides, model)

,prediction_id,rating
0,3342_6_updrs_1_plus_0_months,5.0
4,50423_6_updrs_1_plus_0_months,6.0
0,3342_6_updrs_2_plus_0_months,5.0
4,50423_6_updrs_2_plus_0_months,6.0
0,3342_6_updrs_3_plus_0_months,24.0
4,50423_6_updrs_3_plus_0_months,21.0
0,3342_6_updrs_4_plus_0_months,0.0
4,50423_6_updrs_4_plus_0_months,0.0
0,3342_6_updrs_1_plus_6_months,5.0
4,50423_6_updrs_1_plus_6_months,6.0


## 

In [89]:
# def get_predictions(my_train, test_proteins,test_peptides, model):
#     # Forecast
#     dfff=[]
#     my_train = my_train.fillna(0)
#     train_all303 = mergeall(my_train,test_proteins,test_peptides)
#     ffinal = train_all303.reset_index()
#     tempall=pd.DataFrame()
#     tempall=mytrain[['visit_id']]
#     #ffinal = ffinal.rename(columns={'group_key_x': 'group_key'})
#     #ffinal['visit_id'] = ffinal.index
    
#    # dropcol=['visit_id','patient_id','UniProt','Peptide','row_id','updrs_test']
#     dropcol=['visit_id']
#    # df = ffinal[['visit_id']].drop_duplicates()
#     df = ffinal[['visit_id']]
#     df00 = my_train[['visit_id']]
#     ffinal[['patient_id', 'visit_month']] = ffinal['visit_id'].str.split('_', expand=True)
#     ffinal=ffinal.drop(columns=['visit_id','patient_id'])
#     target=['updrs_3','updrs_1','updrs_2','updrs_4']
#     for u in target:   
#         # Here is where we will save the final results
#         df['result_' + str(u)] = 0
#         # Predict  
#         if u == 'updrs_3':
#             df5 = ffinal
#             last_col = df5.iloc[:, -1]
#             dfsi = df5.iloc[:, :-1]
#             dfsi.insert(0, column='new_col_name', value=last_col)
#             df5=dfsi
#             df5 = pd.DataFrame(scaler.transform(dfsi),columns=dfsi.columns)
#             df['result_' + str(u)] = best_ensemble_model3.predict(df5).tolist()
#         if u == 'updrs_1':
#             df55 = ffinal
#             last_col = df55.iloc[:, -1]
#             dfsib = df55.iloc[:, :-1]
#             dfsib.insert(0, column='new_col_name', value=last_col)
#             df55=dfsib
#             df55['updrs_3']=df['result_updrs_3']
#             df55 = pd.DataFrame(scaler1.transform(dfsib),columns=dfsib.columns)
#             df['result_' + str(u)] = best_ensemble_model1.predict(df55).tolist()
#         if u == 'updrs_2':
#             df555 = ffinal
#             last_col = df555.iloc[:, -1]
#             dfsiba = df555.iloc[:, :-1]
#             dfsiba.insert(0, column='new_col_name', value=last_col)
#             dfsiba.rename(columns={'new_col_name': 'visit_month'}, inplace=True)
#             df555=dfsiba
#             df555['updrs_3']=df['result_updrs_3']
#             df555['updrs_1']=df['result_updrs_1']
#             last_col = df555.iloc[:, -1]
#             dfsiba2 = df555.iloc[:, :-1]
#             dfsiba2.insert(0, column='new_col_name', value=last_col)
#             dfsiba2.rename(columns={'new_col_name': 'updrs_1'}, inplace=True)
#             df555=dfsiba2
#             df555 = pd.DataFrame(scaler2.transform(dfsiba2),columns=dfsiba2.columns)
#             df['result_' + str(u)] = best_ensemble_model2.predict(df555).tolist()
#         if u == 'updrs_4':
#             df['result_' + str(u)] = 0
    
#     a1=model031.predict(df[['result_updrs_3']])
#     a2=model032.predict(df[['result_updrs_3']])
#     a3=model033.predict(df[['result_updrs_3']])
#     a4=model021.predict(df[['result_updrs_2']])
#     a5=model022.predict(df[['result_updrs_2']])
#     a6=model023.predict(df[['result_updrs_2']])
#     a7=model011.predict(df[['result_updrs_1']])
#     a8=model012.predict(df[['result_updrs_1']])
#     a9=model013.predict(df[['result_updrs_1']])
#     a=pd.DataFrame(columns=['dfupdrs3_6','updrs3_12','updrs3_24','updrs2_6','updrs2_12','updrs2_24','updrs1_6','updrs1_12','updrs1_24'])

#     a['updrs1_0']=df[['result_updrs_1']]-1.5
#     a['updrs1_6']=a7-1.5
#     a['updrs1_12']=a8-1
#     a['updrs1_24']=a9-1
#     a['updrs2_0']=df[['result_updrs_2']]
#     a['updrs2_6']=a4
#     a['updrs2_12']=a5
#     a['updrs2_24']=a6
#     a['updrs3_0']=df[['result_updrs_3']]
#     a['updrs3_6']=a1
#     a['updrs3_12']=a2
#     a['updrs3_24']=a3
            
#  # Format for final submission
#     result = pd.DataFrame()
#     result2 = pd.DataFrame()
#     for m in [0, 6, 12, 24]:
#         for u in [1, 2, 3, 4]:
#             temp = df[["visit_id", "result_updrs_" + str(u)]]
#             temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
#             if u!=4:
#                 temp["rating"] = a["updrs" + str(u)+"_"+str(m)]
#             else:
#                 temp["rating"]=0
#             temp = temp [['prediction_id', 'rating']]
#             result = result.append(temp)   
#             tempall["prediction_id"] = tempall["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
#             result2=result2.append(tempall)
#     result = result.drop_duplicates(subset=['prediction_id', 'rating'])
#     result=result.fillna(method='ffill')
#     result=result.fillna(method='bfill')
#     result=result.fillna(temp.mean())
#     result=result.fillna(0)
#     result2 = result.drop_duplicates(subset=['prediction_id'])
    
#     result2['rating']=np.nan
#     visitlist=result[['prediction_id']].values.tolist()
#     result2[result2['prediction_id'].isin(visitlist)][['rating']]=result[["rating"]]
#     result2[['patient_id', 'visit_month']] = result2['visit_id'].str.split('_', expand=True)
#     for i in range(len(result2)):
#         if pd.isna(result2[['rating']].iloc[i,:]):
#             if 'updrs_1' in result2['prediction_id'][i]:
#                 result2['rating'][i]= target_visit_month_medians2.loc[result2['visit_month'][i],'updrs_1']
#             if 'updrs_2' in result2['prediction_id'][i]:
#                 result2['rating'][i]= target_visit_month_medians2.loc[result2['visit_month'][i],'updrs_2']
#             if 'updrs_3' in result2['prediction_id'][i]:
#                 result2['rating'][i]= target_visit_month_medians2.loc[result2['visit_month'][i],'updrs_3']
#             if 'updrs_4' in result2['prediction_id'][i]:
#                 result2['rating'][i]= target_visit_month_medians2.loc[result2['visit_month'][i],'updrs_4']
                
#     result2=result2.fillna(method='ffill')
#     result2=result2.fillna(method='bfill')
#     result2=result2.fillna(temp.mean())
#     result2=result2.fillna(0)
#     result2 = result2[['prediction_id', 'rating']]
#     return result2

In [90]:
# dfff=[]
# my_train = test.fillna(0)
# train_all303 = mergeall(my_train,test_proteins,test_peptides)
# ffinal = train_all303.reset_index()
# tempall=pd.DataFrame()
# tempall=my_train[['visit_id']]
#     #ffinal = ffinal.rename(columns={'group_key_x': 'group_key'})
#     #ffinal['visit_id'] = ffinal.index
    
#    # dropcol=['visit_id','patient_id','UniProt','Peptide','row_id','updrs_test']
# dropcol=['visit_id']
#    # df = ffinal[['visit_id']].drop_duplicates()
# df = ffinal[['visit_id']]
# df00 = my_train[['visit_id']]
# ffinal[['patient_id', 'visit_month']] = ffinal['visit_id'].str.split('_', expand=True)
# ffinal=ffinal.drop(columns=['visit_id','patient_id'])
# target=['updrs_3','updrs_1','updrs_2','updrs_4']
# for u in target:   
#         # Here is where we will save the final results
#     df['result_' + str(u)] = 0
#     if u == 'updrs_3':
#         df5 = ffinal
#         last_col = df5.iloc[:, -1]
#         dfsi = df5.iloc[:, :-1]
#         dfsi.insert(0, column='new_col_name', value=last_col)
#         dfsi.rename(columns={'new_col_name': 'visit_month'}, inplace=True)
#         df5=dfsi
#         df5 = pd.DataFrame(scaler.transform(dfsi),columns=dfsi.columns)
#         df['result_' + str(u)] = best_ensemble_model3.predict(df5).tolist()
#         # Predict  
#     if u == 'updrs_1':
#         df55 = ffinal
#         last_col = df55.iloc[:, -1]
#         dfsib = df55.iloc[:, :-1]
#         dfsib.insert(0, column='new_col_name', value=last_col)
#         dfsib.rename(columns={'new_col_name': 'visit_month'}, inplace=True)
#         df55=dfsib
#         df55['updrs_3']=df['result_updrs_3']
#         df55 = pd.DataFrame(scaler1.transform(dfsib),columns=dfsib.columns)
#         df['result_' + str(u)] = best_ensemble_model1.predict(df55).tolist()
#     if u == 'updrs_2':
#         df555 = ffinal
#         last_col = df555.iloc[:, -1]
#         dfsiba = df555.iloc[:, :-1]
#         dfsiba.insert(0, column='new_col_name', value=last_col)
#         dfsiba.rename(columns={'new_col_name': 'visit_month'}, inplace=True)
#         df555=dfsiba
#         df555['updrs_3']=df['result_updrs_3']
#         df555['updrs_1']=df['result_updrs_1']
#         last_col = df555.iloc[:, -1]
#         dfsiba2 = df555.iloc[:, :-1]
#         dfsiba2.insert(0, column='new_col_name', value=last_col)
#         dfsiba2.rename(columns={'new_col_name': 'updrs_1'}, inplace=True)
#         df555=dfsiba2
#         df555 = pd.DataFrame(scaler2.transform(dfsiba2),columns=dfsiba2.columns)
#         df['result_' + str(u)] = best_ensemble_model2.predict(df555).tolist()
#     if u == 'updrs_4':
#             df['result_' + str(u)] = 0
# a1=model031.predict(df[['result_updrs_3']])
# a2=model032.predict(df[['result_updrs_3']])
# a3=model033.predict(df[['result_updrs_3']])
# a4=model021.predict(df[['result_updrs_2']])
# a5=model022.predict(df[['result_updrs_2']])
# a6=model023.predict(df[['result_updrs_2']])
# a7=model011.predict(df[['result_updrs_1']])
# a8=model012.predict(df[['result_updrs_1']])
# a9=model013.predict(df[['result_updrs_1']])
# a=pd.DataFrame(columns=['updrs3_6','updrs3_12','updrs3_24','updrs2_6','updrs2_12','updrs2_24','updrs1_6','updrs1_12','updrs1_24'])
# a['updrs1_0']=df[['result_updrs_1']]
# a['updrs1_6']=a7
# a['updrs1_12']=a8
# a['updrs1_24']=a9
# a['updrs2_0']=df[['result_updrs_2']]
# a['updrs2_6']=a4
# a['updrs2_12']=a5
# a['updrs2_24']=a6
# a['updrs3_0']=df[['result_updrs_3']]
# a['updrs3_6']=a1
# a['updrs3_12']=a2
# a['updrs3_24']=a3
# result = pd.DataFrame()
# result2 = pd.DataFrame()
# for m in [0, 6, 12, 24]:
#     for u in [1, 2, 3, 4]:
#         temp = df[["visit_id", "result_updrs_" + str(u)]]
#         temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
#         if u!=4:
#             temp["rating"] = a["updrs" + str(u)+"_"+str(m)]
#         else:
#             temp["rating"]=0
#         temp = temp [['prediction_id', 'rating']]
#         result = result.append(temp)
#         tempall["prediction_id"] = tempall["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"


#         result2=result2.append(tempall)
# result = result.drop_duplicates(subset=['prediction_id', 'rating'])
# result=result.fillna(method='ffill')
# result=result.fillna(method='bfill')
# result=result.fillna(temp.mean())
# result=result.fillna(0)
# result2 = result2.drop_duplicates(subset=['prediction_id'])
# result2.loc[result2['prediction_id'].isin(result['prediction_id'].values.tolist()), 'rating'] = result['rating'].values
# result2['rating']=np.nan
# visitlist=result['prediction_id'].values.tolist()

# result2[['patient_id', 'visit_month']] = result2['visit_id'].str.split('_', expand=True)
# for i in range(len(result2)):
#     if pd.isna(result2[['rating']].iloc[i,:])[0]:
#         if 'updrs_1' in result2['prediction_id'].iloc[i]:
#             if result2[['prediction_id']].iloc[i][0][-8]==str(0) or result2[['prediction_id']].iloc[i][0][-8]==str(6):
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_1']
#             elif result2[['prediction_id']].iloc[i][0][-8]==str(2):
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[10+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_1']
#             else:
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[20+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_1']
#         if 'updrs_2' in result2['prediction_id'].iloc[i]:
#             if result2[['prediction_id']].iloc[i][0][-8]==str(0) or result2[['prediction_id']].iloc[i][0][-8]==str(6):
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_2']
#             elif result2[['prediction_id']].iloc[i][0][-8]==str(2):
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[10+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_2']
#             else:
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[20+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_2']
#         if 'updrs_3' in result2['prediction_id'].iloc[i]:
#             if result2[['prediction_id']].iloc[i][0][-8]==str(0) or result2[['prediction_id']].iloc[i][0][-8]==str(6):
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_3']
#             elif result2[['prediction_id']].iloc[i][0][-8]==str(2):
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[10+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_3']
#             else:
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[20+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_3']
#         if 'updrs_4' in result2['prediction_id'].iloc[i]:
#             if result2[['prediction_id']].iloc[i][0][-8]==str(0) or result2[['prediction_id']].iloc[i][0][-8]==str(6):
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_4']
#             elif result2[['prediction_id']].iloc[i][0][-8]==str(2):
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[10+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_4']
#             else:
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[20+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_4']
                
# result2=result2.fillna(method='ffill')
# result2=result2.fillna(method='bfill')
# result2=result2.fillna(temp.mean())
# result2=result2.fillna(0)
# result2 = result2[['prediction_id', 'rating']]
# visitlist22=result[['prediction_id']].values.tolist()
# result2.loc[result2['prediction_id'].isin(result['prediction_id'].values.tolist()), 'rating'] = result['rating'].values
# len(result2)

In [91]:
# for i in range(len(result2)):
#     if pd.isna(result2[['rating']].iloc[i,:])[0]:
#         if 'updrs_1' in result2['prediction_id'].iloc[i]:
#             if result2[['prediction_id']].iloc[i][0][-8]==str(0) or result2[['prediction_id']].iloc[i][0][-8]==str(6):
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_1']
#             elif result2[['prediction_id']].iloc[i][0][-8]==str(2):
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[10+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_1']
#             else:
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[20+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_1']
#         if 'updrs_2' in result2['prediction_id'][i]:
#             if result2[['prediction_id']].iloc[i][0][-8]==str(0) or result2[['prediction_id']].iloc[i][0][-8]==str(6):
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_2']
#             elif result2[['prediction_id']].iloc[i][0][-8]==str(2):
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[10+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_2']
#             else:
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[20+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_2']

# result2


In [92]:
# for i in range(len(result2)):
#     if pd.isna(result2[['rating']].iloc[i,:])[0]:
#         if 'updrs_1' in result2['prediction_id'].iloc[i]:
#             if result2[['prediction_id']].iloc[i][0][-8]==0 or 6:
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_1']
#             if result2[['prediction_id']].iloc[i][0][-8]==2:
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[10+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_1']
#             else:
#                 result2['rating'].iloc[i]= target_visit_month_medians2.loc[20+int(result2[['prediction_id']].iloc[i][0][-8])+int(result2[['visit_month']].iloc[i][0]),'updrs_1']